In [54]:
import json
import requests
from rdflib import Graph, URIRef, Literal, XSD, Namespace, RDF, RDFS
import pandas as pd
from bs4 import BeautifulSoup as BS
from urllib.request import urlopen
from lxml import etree
import re
CT = Namespace('http://myclinicaltrials.org/CT#')
SCHEMA = Namespace('http://schema.org/')

In [238]:
def crawling_data(ID,output):
    url = 'https://clinicaltrials.gov/ct2/show/results/'+ID+'?view=results'
    
    response = urlopen(url)
    soup = BS(response, 'html.parser')
    baseline = soup.find_all("th",class_="de-popFlowLabelCell")
    response = urlopen(url)
    soup = BS(response, 'html.parser')
    dom = etree.HTML(str(soup))
    output['groups'] = []
    output["treatment"] = []
    output["ageOfPatients"] = []
    output["outcome"] = []
    output["outcome_measurements"] = []
    output["age_measurements"] = []
    output["outcome_description"] = []
    output['hazardRatio'] = []
    output['Drugs'] = []
    output['InclusionCriteria'] = []
    output['ExclusionCriteria'] = []
    output['StudyType'] = []

    if (len(baseline)>0):
        groups = []
        number_of_patients = []
        for i in range(0,len(baseline)):
            groups.append(baseline[i].text.strip("\n").strip("\r").strip())
            number_of_patients.append(soup.find_all("td",class_="de-numValue_baselineDataCell")[i].text.strip("\n").strip("\r").strip())
        
        output['groups'] = groups
        output["numberOfPatients"] = []
        if(len(soup.find_all("td", class_="de-numValue_popFlowCell"))>0):          
            for i in range(0,len(baseline)):
                output["numberOfPatients"].append(soup.find_all("td", class_="de-numValue_popFlowCell")[i].text.strip("\r").strip("\n").strip())

 
        treatment = dom.xpath('//*[@id="EXPAND-armGroupDescriptionRow-baseline"]/td[2]/text()')
        if(len(treatment)>0):
#             print('1')
            output["treatment"].append(dom.xpath('//*[@id="EXPAND-armGroupDescriptionRow-baseline"]/td[' + str(i+2)+ ']/text()')[0]
                                       .strip('\n').strip('\r').strip().replace('\n','').replace('\xa0',''))
        else:
#             print('2')
            treatment = dom.xpath('//*[@id="EXPAND-armGroupDescriptionRow-baseline"]/td[2]/p[1]/text()')
            if(len(treatment)>0):
                print(treatment)
                output["treatment"].append(dom.xpath('//*[@id="EXPAND-armGroupDescriptionRow-baseline"]/td['+str(i+2)+']/p[1]/text()')[0]
                                           .strip('\n').strip('\r').strip().replace('\n','').replace('\xa0',''))
          
        output["age_measurements"] = [dom.xpath('//*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[7]/th[1]/div/div[1]/text()'),
                                     dom.xpath('//*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[7]/th[1]/div/div[2]/text()')]
        if(len(output["age_measurements"])>0):
            am1 = dom.xpath('//*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[7]/th[1]/div/div[1]/text()')
            am2 = dom.xpath('//*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[7]/th[1]/div/div[2]/text()')
            if (len(am1)>0 and len(am2)>0):
                output["age_measurements"] = [am1[0].strip('\n').strip('\r').strip().replace('\n','').replace('\xa0',''),
                                     am2[0].strip('\n').strip('\r').strip().replace('\n','').replace('\xa0','')]

                if('count' in output["age_measurements"][0].lower()):
                    print("need to check count")
                else:

                    age = dom.xpath('//*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[9]/td[2]/span/text()')
            #         //*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[9]/td[2]/div[1]
                    if(len(age)>0):
                        for i in range(0,len(baseline)):
                            output["ageOfPatients"].append(dom.xpath('//*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[9]/td[' + str(i+2) + ']/span/text()')
                                                           [0].strip('\n').strip('\r').strip().replace('\n','').replace('\xa0',''))
                    else:
                        age = dom.xpath('//*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[9]/td[2]/div[1]/text()') 
                        for i in range(0,len(baseline)):
                            age = dom.xpath('//*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[9]/td['+str(i+2)+']/div[1]/text()')
                            if len(age)>0:
                                output["ageOfPatients"].append(age[0].strip('\n').strip('\r').strip().replace('\n','').replace('\xa0',''))
            # //*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[9]/td[2]/div[2]
            # //*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[10]/td[2]/div[1]

        outcome = dom.xpath('//*[@id="EXPAND-outcome-data-1"]/table/tbody/tr[6]/td[1]/div[1]/text()')
        
        if(len(outcome) > 0):
            for i in range(0,len(baseline)):
                otc = dom.xpath('//*[@id="EXPAND-outcome-data-1"]/table/tbody/tr[6]/td[' + str(i+1) + ']/div[1]/text()')
                if len(otc)>0:                        
                    output["outcome"].append(otc[0].strip('\n').strip('\r').strip().replace('\n','').replace('\xa0',''))
        else:
            output["outcome"] = []
        output["outcome_measurements"] = [dom.xpath('//*[@id="EXPAND-outcome-data-1"]/table/tbody/tr[5]/th/text()'),
                                     dom.xpath('//*[@id="EXPAND-outcome-data-1"]/table/tbody/tr[5]/th/div/text()')]
        if(len(output["outcome_measurements"]) > 0):
            m1 = dom.xpath('//*[@id="EXPAND-outcome-data-1"]/table/tbody/tr[5]/th/text()')
            m2 = dom.xpath('//*[@id="EXPAND-outcome-data-1"]/table/tbody/tr[5]/th/div/text()')
            if (len(m1)>0 and len(m2)>0):
                output["outcome_measurements"] = [m1[0].strip('\n').strip('\r').strip().replace('\n','').replace('\xa0',''),
                                     m2[0].strip('\n').strip('\r').strip().replace('\n','').replace('\xa0','')]
        outcome_description = dom.xpath('//*[@id="tab-body"]/div[1]/div[4]/div[2]/fieldset[1]/div/table/tr[1]/td/text()')
        if (len(outcome_description)>0):
#             print('0')
            output["outcome_description"] =str(dom.xpath('//*[@id="tab-body"]/div[1]/div[4]/div[2]/fieldset[1]/div/table/tr[1]/td/text()')
            [0]).strip('\n').strip('\r').strip().replace('\n','').replace('\xa0','')

        drug = dom.xpath('//*[@id="tab-body"]/div[1]/div[1]/div/table/tr[4]/td/text()')
        if (len(drug)>0):
            drugs = []
            for i in drug:
                if ': ' in i:
                    drugs.append(i.split(": ",1)[1])
#             output['drug'] = drugs
        
        API_ENDPOINT = 'https://en.wikipedia.org/w/api.php'
        drugs_with_des = {}
        for i in drugs:
            drugs_with_des[i] = {}
            query = i
            params = {
                'action':'query',
                'list':'search',
                'srsearch':query,
                'format':'json',
            }
            drugs_with_des[i]['url'] = requests.get(API_ENDPOINT,params=params).url
            temp = requests.get(API_ENDPOINT,params=params).json()['query']['search']
            if(len(temp)>0):
                des = temp[0]['snippet']
                drugs_with_des[i]['description'] = re.sub(r"<.+?>", "", des)

        output['Drugs'] = drugs_with_des
        hr = dom.xpath('//*[@id="EXPAND-stat-analysis-1-1"]/table/tbody/tr[12]/td/text()')
        if (len(hr)>0):
            hr2 = str(hr[0]).strip('\n').strip()
            output['hazardRatio'] = hr2
    ## get materials from APIs
    studytype_url = 'https://clinicaltrials.gov/api/query/study_fields?expr='+ID+'&fields=studytype&min_rnk=1&max_rnk=&fmt=json'
    studytype = requests.get(studytype_url).json()['StudyFieldsResponse']['StudyFields'][0]['StudyType'][0]
    output['StudyType'] = studytype
#     criteria_url = 'https://clinicaltrials.gov/api/query/full_studies?expr='+ID+'&min_rnk=1&max_rnk=&fmt=json'
    criteria_url = 'https://clinicaltrials.gov/api/query/study_fields?expr='+ID+'&fields=EligibilityCriteria&min_rnk=1&max_rnk=&fmt=json'
    criteria = requests.get(criteria_url).json()['StudyFieldsResponse']['StudyFields'][0]['EligibilityCriteria']

    b = criteria[0].split('\n')
    inclusion = []
    exclusion = []
    for i in range(0,len(b)):
        if b[i].lower().startswith('inclusion criteria'):
            for j in range(i+1,len(b)):
                if b[j].lower().startswith('exclusion criteria'):
                    break
                if b[j] != '':
                    inclusion.append(b[j])
        if b[i].lower().startswith('exclusion criteria'):
            for j in range(i+1,len(b)):
                if b[j] != '':
                    exclusion.append(b[j])
    output['InclusionCriteria'] = inclusion
    output['ExclusionCriteria'] = exclusion
        
    return output






In [239]:
def generate_Graph(ID,output):
    kg = Graph()
    kg.bind('Clinical_Trials', CT)
    kg.bind('schema',SCHEMA)
    ct_uri = URIRef('https://clinicaltrials.gov/ct2/show/'+ID)
    kg.add((ct_uri, RDF.type, SCHEMA.MedicalTrial))
    kg.add((ct_uri, CT.hasID, Literal(ID)))

    if (len(output["StudyType"]) > 0):
#         print(len(output["StudyType"]))
        kg.add((ct_uri, CT.hasStudyType, Literal(output['StudyType'])))
    if (len(output['InclusionCriteria'])>0):
#         print(len(output['InclusionCriteria']))
        for j in output['InclusionCriteria']:
            kg.add((ct_uri, CT.hasInclusionCriteria, Literal(j)))
    if (len(output['ExclusionCriteria'])>0):
#         print(len(output['ExclusionCriteria']))
        for j in output['ExclusionCriteria']:
            kg.add((ct_uri, CT.hasExclusionCriteria, Literal(j)))
    if (len(output['Drugs'])>0):
        for j in output['Drugs']:
            k = URIRef(output['Drugs'][j]['url'])
            kg.add((ct_uri, CT.hasDrug, k))
            kg.add((k, SCHEMA.name, Literal(j)))
            if('description' in output['Drugs'][j]):
                kg.add((k, SCHEMA.description, Literal(output['Drugs'][j]['description'])))

        l = len(output["groups"])

        out_field = ID+'_outcome'
        kg.add((ct_uri, CT.hasOutcome,CT[out_field]))

        treatment_field = ID+'_treamentDescription'
        kg.add((ct_uri, CT.hasTreatmentPlan,CT[treatment_field]))

        nofP_field = ID+'_numberofPatients'
        kg.add((ct_uri, CT.hasNumberofPatients,CT[nofP_field]))

        aofP_field = ID+'_ageOfPatients'
        kg.add((ct_uri, CT.hasAgeOfPatients,CT[aofP_field]))

        if l > 0:
            for i in range(0,len(output["groups"])):
                g = output["groups"][i].replace(' ',"_") + "_Group"
                g0 = g + '_outcome'
                kg.add((CT[out_field], CT.hasGroup, CT[g0]))
                kg.add((CT[g0], CT.hasOutcomeDescription, Literal(output["outcome_description"])))
                if (len(output["outcome_measurements"]) == 2):
                    kg.add((CT[g0], CT.hasMeasurement, Literal(output["outcome_measurements"][0])))
                    kg.add((CT[g0], CT.hasUnit, Literal(output["outcome_measurements"][1])))
                if (len(output['outcome']) == l):
                    kg.add((CT[g0],SCHEMA.number,Literal(output['outcome'][i])))

                g1 = g + '_treatment'
                kg.add((CT[treatment_field], CT.hasTreatment,CT[g1]))
                if (len(output["treatment"]) == l):
                    kg.add((CT[g1], CT.hasGroup, Literal(g)))
                    kg.add((CT[g1], CT.hasdescription, Literal(output["treatment"][i])))

                g2 = g + '_numberOfPatients'
                kg.add((CT[nofP_field], CT.hasTreatment,CT[g2]))
                if (len(output["numberOfPatients"]) == l):
                    kg.add((CT[g2], CT.hasGroup, Literal(g)))
                    kg.add((CT[g2], SCHEMA.number, Literal(output["numberOfPatients"][i])))
                    kg.add((CT[g2], CT.hasUnit, Literal("person")))

                g3 = g + '_ageOfPatients'
                kg.add((CT[aofP_field], CT.hasAge,CT[g3]))
                if (len(output["ageOfPatients"]) == l):
                    kg.add((CT[g3], CT.hasGroup, Literal(g)))
                    kg.add((CT[g3], SCHEMA.number, Literal(output["ageOfPatients"][i])))
                if (len(output["age_measurements"]) == 2):
                    kg.add((CT[g3], CT.hasType, Literal(output["age_measurements"][0])))
                    kg.add((CT[g3], CT.hasUnit, Literal(output["age_measurements"][1])))


            hazardRatio = ID + '_hazardRatio'
            if(output["hazardRatio"])!=[]:
                kg.add((ct_uri, CT.hasHazardRatio, CT[hazardRatio]))
                kg.add((CT[hazardRatio], SCHEMA.number, Literal(output["hazardRatio"])))

    kg.serialize('sample_graphs/'+ID+'.ttl', format="turtle")

In [244]:
ID = 'NCT04495244'
output = {}
res = crawling_data(ID,output)

# print(res)
generate_Graph(ID,res)

In [257]:
import os
a = []
directory = 'sample_graphs'
for filename in os.listdir(directory):
    file_size = os.path.getsize('sample_graphs/'+filename)
    if file_size<=1000:

        a.append(filename.strip('.ttl'))
len(a)


87

## Get 202 clinical trials of Brest Cancer

In [179]:
hundred_url = 'https://clinicaltrials.gov/api/query/study_fields?expr=breast+cancer&fields=NCTId&min_rnk=1&max_rnk=200&fmt=json'
ID = requests.get(hundred_url).json()['StudyFieldsResponse']['StudyFields']
IDs = []
for id in ID:
    IDs.append(id['NCTId'][0])


In [254]:
len(a)

89

In [255]:
count = 0
for i in a:
    output = {}
    generate_Graph(i,crawling_data(i,output))
    print(i,count+1)
    count += 1

NCT03662633 1
NCT02493569 2
NCT03598660 3
NCT04516330 4
NCT04167605 5
NCT04590560 6
NCT01299623 7
NCT04450264 8
NCT01064349 9
NCT01521741 10
NCT03681418 11
NCT01789684 12
NCT04948983 13
NCT01257932 14
NCT01879189 15
NCT00897962 16
NCT00656604 17
NCT02810093 18
NCT03390608 19
NCT01916837 20
NCT04980989 21
NCT02934126 22
NCT00416975 23
NCT02954900 24
NCT02808598 25
NCT01914614 26
NCT01366248 27
NCT01434420 28
NCT04488614 29
NCT04906330 30
NCT01552655 31
NCT03436069 32
NCT04720508 33
NCT01403779 34
NCT04778202 35
NCT03800355 36
NCT01521676 37
NCT04258085 38
NCT03429504 39
NCT01391806 40
NCT03069742 41
NCT03461172 42
NCT03480659 43
NCT04976556 44
NCT02688725 45
need to check count
NCT04090164 46
NCT00078832 47
NCT03257917 48
NCT00897468 49
NCT04304404 50
NCT01972048 51
NCT01320488 52
NCT04033965 53
NCT03127332 54
NCT00003906 55
NCT02212834 56
NCT02303145 57
NCT00416403 58
NCT02094495 59
NCT00393341 60
NCT00225927 61
NCT02480933 62
NCT01335152 63
NCT02710058 64
NCT03300349 65
NCT04348955 66

In [173]:

for i in range(195,len(IDs)):
    print(IDs[i],i+1)
    output = {}
    generate_Graph(IDs[i],crawling_data(IDs[i],output))
    


NCT00847171 196
NCT02964234 197
NCT01220960 198
NCT01468246 199
NCT00991653 200


In [182]:
kg = Graph()
kg.bind('Clinical_Trials', CT)
kg.bind('schema',SCHEMA)
ct_uri = URIRef('https://clinicaltrials.gov/ct2/show/'+ID)
kg.add((ct_uri, RDF.type, SCHEMA.MedicalTrial))
kg.add((ct_uri, CT.hasID, Literal(ID)))
for i in output:
    if (len(out["StudyType"]) > 0):
        kg.add((ct_uri, CT.hasStudyType, Literal(output['StudyType'])))
    if (i == 'InclusionCriteria' and len(output['InclusionCriteria'])>0):
        for j in output[i]:
            kg.add((ct_uri, CT.hasInclusionCriteria, Literal(j)))
    if (i == 'ExclusionCriteria' and len(output['ExclusionCriteria'])>0):
        for j in output[i]:
            kg.add((ct_uri, CT.hasExclusionCriteria, Literal(j)))
    if (i == 'Drugs' and len(output['Drugs'])>0):
        for j in output[i]:
            k = URIRef(output[i][j]['url'])
            kg.add((ct_uri, CT.hasDrug, k))
            kg.add((k, SCHEMA.name, Literal(j)))
            kg.add((k, SCHEMA.description, Literal(output[i][j]['description'])))
    
    l = len(output["groups"])

    out_field = ID+'_outcome'
    kg.add((ct_uri, CT.hasOutcome,CT[out_field]))
    
    treatment_field = ID+'_treamentDescription'
    kg.add((ct_uri, CT.hasTreatmentPlan,CT[treatment_field]))

    nofP_field = ID+'_numberofPatients'
    kg.add((ct_uri, CT.hasNumberofPatients,CT[nofP_field]))
    
    aofP_field = 'NCT04199078_ageOfPatients'
    kg.add((ct_uri, CT.hasAge,CT[aofP_field]))
    
    if l > 0:
        for i in range(0,len(outcome["groups"])):
            g = outcome["groups"][i].replace(' ',"_") + "_Group"
            g0 = g + "_outcome"
            kg.add((CT[out_field], CT.hasGroup, CT(g0)))
            kg.add((CT[g0], CT.hasOutcomeDescription, Literal(output["outcome_description"])))
            if (len(output["outcome_measurements"]) == 2):
                kg.add((CT[g0], CT.hasMeasurement, Literal(output["outcome_measurements"][0])))
                kg.add((CT[g0], CT.hasUnit, Literal(output["outcome_measurements"][1])))
            kg.add((CT(g0),SCHEMA.number,Literal(output['outcome'][i])))

            g1 = g + '_treatment'
            kg.add((CT[treatment_field], CT.hasTreatment,CT[g1]))
            if (len(output["treatment"]) == l):
                kg.add((CT[g1], CT.hasGroup, Literal(g)))
                kg.add((CT[g1], CT.hasdescription, Literal(output["treatment"][i])))

            g2 = g + '_numberOfPatients'
            kg.add((CT[nofP_field], CT.hasTreatment,CT[g2]))
            if (len(output["numberOfPatients"]) == l):
                kg.add((CT[g2], CT.hasGroup, Literal(g)))
                kg.add((CT[g2], SCHEMA.number, Literal(output["numberOfPatients"][i])))
                kg.add((CT[g2], CT.hasUnit, Literal("person")))
    
            g3 = g + '_ageOfPatients'
            kg.add((CT[aofP_field], CT.hasAgeofPatients,CT[g3]))
            if (len(output["age_measurements"]) == l):
                kg.add((CT[g3], CT.hasGroup, Literal(g)))
                kg.add((CT[g3], SCHEMA.number, Literal(output["ageOfPatients"][i])))
            if (len(output["age_measurements"][0]) == 2):
                kg.add((CT[g3], CT.hasType, Literal(output["age_measurements"][0])))
                kg.add((CT[g3], CT.hasUnit, Literal(output["age_measurements"][1])))


        hazardRatio = ID + '_hazardRatio'
        kg.add((ct_uri, CT.hasHazardRatio, CT[hazardRatio]))
        kg.add((CT[hazardRatio], SCHEMA.number, Literal(output["hazardRatio"])))

kg.serialize('100_graphs/'+ID'.ttl', format="turtle")    

SyntaxError: invalid syntax (<ipython-input-182-8bc31adb950d>, line 59)

In [84]:
url = 'https://clinicaltrials.gov/ct2/show/results/NCT01740427?view=results'
    
response = urlopen(url)
soup = BS(response, 'html.parser')
baseline = soup.find_all("th",class_="de-popFlowLabelCell")
response = urlopen(url)
soup = BS(response, 'html.parser')
dom = etree.HTML(str(soup))

hr = str(dom.xpath('//*[@id="EXPAND-stat-analysis-1-1"]/table/tbody/tr[12]/td/text()')[0]).strip('\n').strip()
hr

'0.576'

In [ ]:
NCT03093350 = dict()
studytype_url = 'https://clinicaltrials.gov/api/query/study_fields?expr=NCT03093350&fields=studytype&min_rnk=1&max_rnk=&fmt=json'
studytype = requests.get(studytype_url).json()['StudyFieldsResponse']['StudyFields'][0]['StudyType'][0]
NCT03093350['StudyType'] = studytype
criteria_url = 'https://clinicaltrials.gov/api/query/full_studies?expr=NCT04176354&min_rnk=1&max_rnk=&fmt=json'
criteria = requests.get(criteria_url).json()['FullStudiesResponse']['FullStudies'][0]['Study']['ProtocolSection']['EligibilityModule']['EligibilityCriteria']

b = criteria.split('\n')
inclusion = []
exclusion = []
for i in range(0,len(b)):
    if b[i] == 'Inclusion Criteria:':
        for j in range(i+1,len(b)):
            if b[j] == 'Exclusion Criteria:':
                break
            if b[j] != '':
                inclusion.append(b[j])
    if b[i] == 'Exclusion Criteria:':
        for j in range(i+1,len(b)):
            if b[j] != '':
                exclusion.append(b[j])
NCT03093350['InclusionCriteria'] = inclusion
NCT03093350['ExclusionCriteria'] = exclusion

drug_url = 'https://clinicaltrials.gov/api/query/study_fields?expr=NCT04176354&fields=ArmGroupInterventionName&min_rnk=1&max_rnk=&fmt=json'
drug = requests.get(drug_url).json()['StudyFieldsResponse']['StudyFields'][0]['ArmGroupInterventionName']
drug
drugs = [i.strip("Drug:").strip("'").strip() for i in drug] 
drugs2 = set()
for i in drugs:
    for j in i.split(" "):
        drugs2.add(j)
# drugs = list(dict.fromkeys(drugs))
drugs2.remove('an')
drugs2.remove('+')
drugs2.remove('aromatase')
drugs2.remove('inhibito')

API_ENDPOINT = 'https://en.wikipedia.org/w/api.php'
drugs_with_des = {}
for i in drugs2:
    drugs_with_des[i] = {}
    query = i
    params = {
        'action':'query',
        'list':'search',
        'srsearch':query,
        'format':'json',
    }
    drugs_with_des[i]['url'] = requests.get(API_ENDPOINT,params=params).url
    drugs_with_des[i]['description'] = requests.get(API_ENDPOINT,params=params).json()\
    ['query']['search'][0]['snippet']
drugs_with_des
NCT03093350['Drugs'] = drugs_with_des



url = "https://clinicaltrials.gov/ct2/show/results/NCT03093350?view=results"
response = urlopen(url)
soup = BS(response, 'html.parser')
baseline = soup.find_all("th",class_="de-popFlowLabelCell")
groups = []
number_of_patients = []
for i in range(0,len(baseline)):
    groups.append(baseline[i].text.strip("\n").strip("\r").strip())
    number_of_patients.append(soup.find_all("td",class_="de-numValue_baselineDataCell")[i].text.strip("\n").strip("\r").strip())

response = urlopen(url)
soup = BS(response, 'html.parser')
dom = etree.HTML(str(soup))


monaleesa2["numberOfPatients"] = []
monaleesa2["numberOfPatients"].append(soup.find_all("td", class_="de-numValue_popFlowCell")[0].text.strip("\r").strip("\n").strip())
monaleesa2["numberOfPatients"].append(soup.find_all("td", class_="de-numValue_popFlowCell")[1].text.strip("\r").strip("\n").strip())
monaleesa2["outcome"] = []
monaleesa2["outcome"].append(soup.find_all("td", class_="de-numValue_outcomeDataCell")[2].text.strip('\n').strip().split(" ")[0])
monaleesa2["outcome"].append(soup.find_all("td", class_="de-numValue_outcomeDataCell")[3].text.strip('\n').strip().split(" ")[0])
monaleesa2["treatment"] = []
monaleesa2["treatment"].append(soup.find_all("td",class_="de-outcomeLabelCell")[7].text.strip('\n').strip('\r').strip())
monaleesa2["treatment"].append(soup.find_all("td",class_="de-outcomeLabelCell")[8].text.strip('\n').strip('\r').strip())
monaleesa2["ageOfPatients"] = []
monaleesa2["ageOfPatients"].append(soup.find_all("td",class_="de-numValue_baselineDataCell")[11].text.strip('\n').strip('\r').strip().split(" ")[0])
monaleesa2["ageOfPatients"].append(soup.find_all("td",class_="de-numValue_baselineDataCell")[12].text.strip('\n').strip('\r').strip().split(" ")[0])
monaleesa2["hazardRatio"] = soup.find_all("td",class_="de-outcomeDataCell")[15].text.strip('\n').strip('\r').strip()

monaleesa2['PFS'] = [dom.xpath('//*[@id="EXPAND-outcome-data-1"]/table/tbody/tr[6]/td[1]/div[1]/text()')[0]
       .strip('\n').strip('\r').strip().replace('\n','').replace('\xa0',''),
                  dom.xpath('//*[@id="EXPAND-outcome-data-1"]/table/tbody/tr[6]/td[2]/div[1]/text()')[0].strip('\n').strip().strip('\n').strip()]
monaleesa2["outcome_measurements"] = [dom.xpath('//*[@id="EXPAND-outcome-data-1"]/table/tbody/tr[5]/th/text()')[0]
       .strip('\n').strip('\r').strip().replace('\n','').replace('\xa0',''),
                             dom.xpath('//*[@id="EXPAND-outcome-data-1"]/table/tbody/tr[5]/th/div/text()')[0]
       .strip('\n').strip('\r').strip().replace('\n','').replace('\xa0','')]
monaleesa2["age_measurements"] = [dom.xpath('//*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[7]/th[1]/div/div[1]/text()')[0]
       .strip('\n').strip('\r').strip().replace('\n','').replace('\xa0',''),
                             dom.xpath('//*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[7]/th[1]/div/div[2]/text()')[0]
       .strip('\n').strip('\r').strip().replace('\n','').replace('\xa0','')]





NCT03093350["numberOfPatients"] = str(dom.xpath('//*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[4]/td/text()')[0]).strip('\n').strip('\r').strip().replace('\n','').replace('\xa0','')
    
NCT03093350["groupTitle"] = str(dom.xpath('//*[@id="tab-body"]/div[1]/div[2]/div[2]/table[2]/tbody/tr[1]/th/text()')[0]).strip('\n').strip('\r').strip().replace('\n','').replace('\xa0','')

NCT03093350["groupDescription"] = str(dom.xpath('//*[@id="EXPAND-armGroupDescriptionRow-baseline"]/td[2]//p/text()'))

NCT03093350["ageOfPatients"] = str(dom.xpath('//*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[9]/td[2]/span/text()')[0]).strip('\n').strip('\r').strip().replace('\n','').replace('\xa0','')

NCT03093350["PFS"] = str(dom.xpath('//*[@id="EXPAND-outcome-data-2"]/table/tbody/tr[6]/td/div[1]/text()')[0]).strip('\n').strip('\r').strip().replace('\n','').replace('\xa0','')    

NCT03093350["PFS_unit"] = str(dom.xpath('//*[@id="EXPAND-outcome-data-2"]/table/tbody/tr[5]/th/div/text()')[0]).strip('\n').strip('\r').strip().replace('\n','').replace('\xa0','') 

NCT03093350["OAS"] = str(dom.xpath('//*[@id="EXPAND-outcome-data-3"]/table/tbody/tr[6]/td/div[1]/text()')[0]).strip('\n').strip('\r').strip().replace('\n','').replace('\xa0','')    

NCT03093350["OAS_unit"] = str(dom.xpath('//*[@id="EXPAND-outcome-data-3"]/table/tbody/tr[5]/th/div/text()')[0]).strip('\n').strip('\r').strip().replace('\n','').replace('\xa0','') 

kg_NCT03093350 = Graph()
kg_NCT03093350.bind('Clinical_Trials', CT)
kg_NCT03093350.bind('schema',SCHEMA)
kg_NCT03093350.add((ct_uri, CT.hasID, Literal('NCT03093350')))
            
ct_uri = URIRef('https://clinicaltrials.gov/ct2/show/NCT03093350')
kg_NCT03093350.add((ct_uri, RDF.type, SCHEMA.MedicalTrial))
title = str(NCT03093350["groupTitle"].replace(' ','_'))
kg_NCT03093350.add((ct_uri, CT.hasGroup, CT[title]))
kg_NCT03093350.add((CT[title], CT.hasDescription, Literal(NCT03093350["groupDescription"])))

kg_NCT03093350.add((ct_uri, CT.hasNumberofPatients, Literal(NCT03093350["numberOfPatients"])))
kg_NCT03093350.add((ct_uri, CT.hasMedianAge, Literal(NCT03093350["ageOfPatients"])))
kg_NCT03093350.add((ct_uri, CT.hasPFS, CT['NCT03093350_PFS']))
kg_NCT03093350.add((CT['NCT03093350_PFS'], CT.hasUnit, Literal(NCT03093350["PFS_unit"])))
kg_NCT03093350.add((CT['NCT03093350_PFS'], SCHEMA.number, Literal(NCT03093350["PFS"])))
kg_NCT03093350.add((ct_uri, CT.hasOAS, CT['NCT03093350_OAS']))
kg_NCT03093350.add((CT['NCT03093350_OAS'], CT.hasUnit, Literal(NCT03093350["OAS_unit"])))
kg_NCT03093350.add((CT['NCT03093350_OAS'], SCHEMA.number, Literal(NCT03093350["OAS"])))  

for i in NCT03093350:
    kg_NCT03093350.add((ct_uri, CT.hasStudyType, Literal(NCT03093350['StudyType'])))
    if i == 'InclusionCriteria':
        for j in NCT03093350[i]:
            kg_NCT03093350.add((ct_uri, CT.hasInclusionCriteria, Literal(j)))
    if i == 'ExclusionCriteria':
        for j in NCT03093350[i]:
            kg_NCT03093350.add((ct_uri, CT.hasExclusionCriteria, Literal(j)))
    if i == 'Drugs':
        for j in NCT03093350[i]:
            k = URIRef(NCT03093350[i][j]['url'])
            kg_NCT03093350.add((ct_uri, CT.hasDrug, k))
            kg_NCT03093350.add((k, SCHEMA.name, Literal(j)))
            kg_NCT03093350.add((k, SCHEMA.description, Literal(NCT03093350[i][j]['description'])))
            
kg_NCT03093350.serialize('new_graphs/NCT03093350.ttl',format="turtle")



## Extract all critera

In [127]:
paloma2 = dict()
studytype_url = 'https://clinicaltrials.gov/api/query/field_values?expr=paloma2&field=StudyType&fmt=json'
studytype = requests.get(studytype_url).json()['FieldValuesResponse']['FieldValues'][0]['FieldValue']
paloma2['StudyType'] = studytype
criteria_url = 'https://clinicaltrials.gov/api/query/full_studies?expr=paloma2&min_rnk=1&max_rnk=&fmt=json'
criteria = requests.get(criteria_url).json()['FullStudiesResponse']['FullStudies'][0]['Study']['ProtocolSection']['EligibilityModule']['EligibilityCriteria']
criteria

'Inclusion Criteria:\n\nWoman between the ages of 30 and 65 (both inclusive).\nAble to read and understand the Patient Information Sheet and the Informed Consent form.\nAcceptance in the participation of the essay and signature of the Informed Consent form.\nASC-US or LSIL or AG-US cytological result (maximum 3 months prior to inclusion) with concordant colposcopic image (performed at the screening visit).\nHigh risk HPV positive by PCR performed at screening visit.\nIs able, at the discretion of the researcher, to comply with the requirements of the trial and without hindrance to follow the instructions and assessments throughout it.\n\nExclusion Criteria:\n\nClinically relevant immune system alterations, or any other autoimmune disease or in treatment with immunosuppressants.\nNon-diagnosed abnormal genital hemorrhage (during the 6 months prior to the screening visit).\nTo had been vaccinated against HPV.\nOther symptomatic vulvovaginal infections.\nSurgical cervical excision in the 

In [128]:
a = criteria.split('\n')
inclusion = []
exclusion = []
for i in range(0,len(a)):
    if a[i] == 'Inclusion Criteria:':
        for j in range(i+1,len(a)):
            if a[j] == 'Exclusion Criteria:':
                break
            if a[j] != '':
                inclusion.append(a[j])
    if a[i] == 'Exclusion Criteria:':
        for j in range(i+1,len(a)):
            if a[j] != '':
                exclusion.append(a[j])
paloma2['InclusionCriteria'] = inclusion
paloma2['ExclusionCriteria'] = exclusion
paloma2

{'StudyType': 'Interventional',
 'InclusionCriteria': ['Woman between the ages of 30 and 65 (both inclusive).',
  'Able to read and understand the Patient Information Sheet and the Informed Consent form.',
  'Acceptance in the participation of the essay and signature of the Informed Consent form.',
  'ASC-US or LSIL or AG-US cytological result (maximum 3 months prior to inclusion) with concordant colposcopic image (performed at the screening visit).',
  'High risk HPV positive by PCR performed at screening visit.',
  'Is able, at the discretion of the researcher, to comply with the requirements of the trial and without hindrance to follow the instructions and assessments throughout it.'],
 'ExclusionCriteria': ['Clinically relevant immune system alterations, or any other autoimmune disease or in treatment with immunosuppressants.',
  'Non-diagnosed abnormal genital hemorrhage (during the 6 months prior to the screening visit).',
  'To had been vaccinated against HPV.',
  'Other symptom

In [129]:
drug_url = 'https://clinicaltrials.gov/api/query/study_fields?expr=paloma-2&fields=ArmGroupInterventionName&min_rnk=2&max_rnk=2&fmt=json'
drug = requests.get(drug_url).json()['StudyFieldsResponse']['StudyFields'][0]['ArmGroupInterventionName']
drug
drugs = [i.strip("Drug:").strip("'").strip() for i in drug] 
drugs = list(dict.fromkeys(drugs))
drugs

['PD-0332991', 'Letrozole', 'Placebo']

In [130]:
API_ENDPOINT = 'https://www.wikidata.org/w/api.php'
drugs_with_des = {}
for i in drugs:
    query = i
    params = {
        'action':'wbsearchentities',
        'format':'json',
        'language':'en',
        'search':query,
    }
    drugs_with_des[i] = requests.get(API_ENDPOINT,params=params).json()['search'][0]['description']

In [131]:
drugs
API_ENDPOINT = 'https://en.wikipedia.org/w/api.php'
query = 'Placebo'
params = {
    'action':'query',
    'list':'search',
    'srsearch':query,
    'format':'json',
}
a = requests.get(API_ENDPOINT,params=params).json()['query']['search'][0]['snippet']
a

'A <span class="searchmatch">placebo</span> (/pləˈsiːboʊ/ plə-SEE-boh) is a substance or treatment which is designed to have no therapeutic value. Common <span class="searchmatch">placebos</span> include inert tablets'

In [132]:
API_ENDPOINT = 'https://en.wikipedia.org/w/api.php'
drugs_with_des = {}
for i in drugs:
    drugs_with_des[i] = {}
    query = i
    params = {
        'action':'query',
        'list':'search',
        'srsearch':query,
        'format':'json',
    }
    drugs_with_des[i]['url'] = requests.get(API_ENDPOINT,params=params).url
    drugs_with_des[i]['description'] = requests.get(API_ENDPOINT,params=params).json()\
    ['query']['search'][0]['snippet']
drugs_with_des

{'PD-0332991': {'url': 'https://en.wikipedia.org/w/api.php?action=query&list=search&srsearch=PD-0332991&format=json',
  'description': 'Kalous, O; Cohen, DJ; Desai, AJ; Ginther, C; Atefi, M; et\xa0al. (2009). &quot;<span class="searchmatch">PD</span> <span class="searchmatch">0332991</span>, a selective cyclin D kinase 4/6 inhibitor, preferentially inhibits proliferation'},
 'Letrozole': {'url': 'https://en.wikipedia.org/w/api.php?action=query&list=search&srsearch=Letrozole&format=json',
  'description': '<span class="searchmatch">Letrozole</span>, sold under the brand name Femara among others, is an aromatase inhibitor which is used in the treatment of hormonally-responsive breast cancer'},
 'Placebo': {'url': 'https://en.wikipedia.org/w/api.php?action=query&list=search&srsearch=Placebo&format=json',
  'description': 'A <span class="searchmatch">placebo</span> (/pləˈsiːboʊ/ plə-SEE-boh) is a substance or treatment which is designed to have no therapeutic value. Common <span class="sea

In [133]:
API_ENDPOINT = 'https://en.wikipedia.org/w/api.php'
query = 'Letrozole'
params = {
        'action':'query',
        'list':'search',
        'srsearch':query,
        'format':'json',
    }
res = requests.get(API_ENDPOINT,params=params).url
res

'https://en.wikipedia.org/w/api.php?action=query&list=search&srsearch=Letrozole&format=json'

In [134]:
paloma2['Drugs'] = drugs_with_des
paloma2

{'StudyType': 'Interventional',
 'InclusionCriteria': ['Woman between the ages of 30 and 65 (both inclusive).',
  'Able to read and understand the Patient Information Sheet and the Informed Consent form.',
  'Acceptance in the participation of the essay and signature of the Informed Consent form.',
  'ASC-US or LSIL or AG-US cytological result (maximum 3 months prior to inclusion) with concordant colposcopic image (performed at the screening visit).',
  'High risk HPV positive by PCR performed at screening visit.',
  'Is able, at the discretion of the researcher, to comply with the requirements of the trial and without hindrance to follow the instructions and assessments throughout it.'],
 'ExclusionCriteria': ['Clinically relevant immune system alterations, or any other autoimmune disease or in treatment with immunosuppressants.',
  'Non-diagnosed abnormal genital hemorrhage (during the 6 months prior to the screening visit).',
  'To had been vaccinated against HPV.',
  'Other symptom

In [135]:
# from tabula import read_pdf
# table = read_pdf('paloma2.pdf',pages = 4,output_format='json')

In [136]:
def get_vals(test_dict, key): 
    for i, j in test_dict.items(): 
        if i == key: 
            yield (j) 
        yield from [] if not isinstance(j, dict) else get_vals(j, key)


In [137]:
# data = []
# for i in table[0]['data']:
#     for j in i:
#         data.append(list(get_vals(j,'text')))
# data = list(filter(lambda a: a != [''],data))

In [138]:
# paloma2['data'] = data

In [139]:
# outcome_url = "https://clinicaltrials.gov/api/query/study_fields?expr=paloma2&fields=OutcomeMeasurementValue&min_rnk=1&max_rnk=&fmt=json"
# outcome = requests.get(outcome_url).json()["StudyFieldsResponse"]["StudyFields"][1]["OutcomeMeasurementValue"][:2]

# treatment_url = "https://clinicaltrials.gov/api/query/study_fields?expr=paloma2&fields=ArmGroupDescription&min_rnk=1&max_rnk=&fmt=json"
# treatment = requests.get(treatment_url).json()["StudyFieldsResponse"]["StudyFields"][1]["ArmGroupDescription"]

# numberOfPatients_url = "https://clinicaltrials.gov/api/query/study_fields?expr=paloma2&fields=BaselineDenomCountValue&min_rnk=1&max_rnk=&fmt=json"
# numberOfPatients = requests.get(numberOfPatients_url).json()["StudyFieldsResponse"]["StudyFields"][1]["BaselineDenomCountValue"]

# ageOfPatients_url = "https://clinicaltrials.gov/api/query/study_fields?expr=paloma2&fields=BaselineMeasurementValue&min_rnk=1&max_rnk=&fmt=json"
# ageOfPatients = requests.get(ageOfPatients_url).json()["StudyFieldsResponse"]["StudyFields"][1]["BaselineMeasurementValue"][:3]
# ageOfPatients

# hazardRatio_url = "https://clinicaltrials.gov/api/query/study_fields?expr=paloma2&fields=OutcomeAnalysisParamValue&min_rnk=1&max_rnk=&fmt=json"
# hazardRatio = requests.get(hazardRatio_url).json()["StudyFieldsResponse"]["StudyFields"][1]["OutcomeAnalysisParamValue"][0]
# hazardRatio

In [140]:
outcome_url = "https://clinicaltrials.gov/api/query/study_fields?expr=paloma2&fields=OutcomeMeasurementValue&min_rnk=1&max_rnk=&fmt=json"
outcome = requests.get(outcome_url).json()["StudyFieldsResponse"]["StudyFields"][1]["OutcomeMeasurementValue"][:2]


In [141]:
treatment_url = "https://clinicaltrials.gov/api/query/study_fields?expr=paloma2&fields=ArmGroupDescription&min_rnk=1&max_rnk=&fmt=json"
treatment = requests.get(treatment_url).json()["StudyFieldsResponse"]["StudyFields"][1]["ArmGroupDescription"]
treatment

['PD-0332991, 125mg, orally once daily on Day 1 to Day 21 of every 28-day cycle followed by 7 days off treatment in combination with Letrozole, 2.5mg, orally once daily (continuously).',
 'Placebo, 125mg, orally once daily on Day 1 to Day 21 of every 28-day cycle followed by 7 days off treatment in combination with Letrozole, 2.5mg, orally once daily (continuously).']

In [142]:
numberOfPatients_url = "https://clinicaltrials.gov/api/query/study_fields?expr=paloma2&fields=BaselineDenomCountValue&min_rnk=1&max_rnk=&fmt=json"
numberOfPatients = requests.get(numberOfPatients_url).json()["StudyFieldsResponse"]["StudyFields"][1]["BaselineDenomCountValue"]
numberOfPatients

['444', '222', '666']

In [143]:
ageOfPatients_url = "https://clinicaltrials.gov/api/query/study_fields?expr=paloma2&fields=BaselineMeasurementValue&min_rnk=1&max_rnk=&fmt=json"
ageOfPatients = requests.get(ageOfPatients_url).json()["StudyFieldsResponse"]["StudyFields"][1]["BaselineMeasurementValue"][:3]
ageOfPatients

['61.7', '60.6', '61.3']

In [144]:
hazardRatio_url = "https://clinicaltrials.gov/api/query/study_fields?expr=paloma2&fields=OutcomeAnalysisParamValue&min_rnk=1&max_rnk=&fmt=json"
hazardRatio = requests.get(hazardRatio_url).json()["StudyFieldsResponse"]["StudyFields"][1]["OutcomeAnalysisParamValue"][0]
hazardRatio

'0.576'

In [145]:
# paloma2["outcome"] = outcome
# paloma2["treatment"] = treatment
# paloma2["numberOfPatients"] = numberOfPatients
# paloma2["ageOfPatients"] = ageOfPatients
# paloma2["hazardRatio"] = hazardRatio
paloma2["hasPercentageOfWhite"] = [77.5,77.5]
paloma2["hasPercentageOfPatientsWithVisceralDisease"] = [48.2,49.5]
paloma2["hasPercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy"] = [48,49.1]
paloma2["hasPercentageOfPatientsWithNewlyDiagonosedAdavancedDisease"] = 37.2
paloma2

{'StudyType': 'Interventional',
 'InclusionCriteria': ['Woman between the ages of 30 and 65 (both inclusive).',
  'Able to read and understand the Patient Information Sheet and the Informed Consent form.',
  'Acceptance in the participation of the essay and signature of the Informed Consent form.',
  'ASC-US or LSIL or AG-US cytological result (maximum 3 months prior to inclusion) with concordant colposcopic image (performed at the screening visit).',
  'High risk HPV positive by PCR performed at screening visit.',
  'Is able, at the discretion of the researcher, to comply with the requirements of the trial and without hindrance to follow the instructions and assessments throughout it.'],
 'ExclusionCriteria': ['Clinically relevant immune system alterations, or any other autoimmune disease or in treatment with immunosuppressants.',
  'Non-diagnosed abnormal genital hemorrhage (during the 6 months prior to the screening visit).',
  'To had been vaccinated against HPV.',
  'Other symptom

In [146]:
from rdflib import Graph, URIRef, Literal, XSD, Namespace, RDF, RDFS
import pandas as pd
import json

In [147]:
CT = Namespace('http://myclinicaltrials.org/CT#')
SCHEMA = Namespace('http://schema.org/')

In [148]:
# kg = Graph()
# kg.bind('Clinical_Trials', CT)
# kg.bind('schema',SCHEMA)

## crawling outcome from html page

In [149]:

from bs4 import BeautifulSoup as BS
from urllib.request import urlopen

In [150]:
# url = "https://clinicaltrials.gov/ct2/show/results/NCT01740427?cond=paloma2&draw=2&rank=1&view=results"
# html = urlopen(url)
# soup = BS(html, 'lxml')
# title = soup.title
# print(title)

In [163]:
url = "https://clinicaltrials.gov/ct2/show/results/NCT01740427?view=results"
response = urlopen(url)
soup = BS(response, 'html.parser')
dom = etree.HTML(str(soup))
baseline = soup.find_all("th",class_="de-popFlowLabelCell")
groups = []
number_of_patients = []
for i in range(0,len(baseline)):
    groups.append(baseline[i].text.strip("\n").strip("\r").strip())
    number_of_patients.append(soup.find_all("td",class_="de-numValue_baselineDataCell")[i].text.strip("\n").strip("\r").strip())
paloma2["groups"] = groups
paloma2["numberOfPatients"] = number_of_patients
paloma2["ageOfPatients"] = [dom.xpath('//*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[9]/td[2]/span/text()')[0]
       .strip('\n').strip('\r').strip().replace('\n','').replace('\xa0',''),
                               dom.xpath('//*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[9]/td[3]/span/text()')[0]
       .strip('\n').strip('\r').strip().replace('\n','').replace('\xa0','')]


paloma2

{'StudyType': 'Interventional',
 'InclusionCriteria': ['Woman between the ages of 30 and 65 (both inclusive).',
  'Able to read and understand the Patient Information Sheet and the Informed Consent form.',
  'Acceptance in the participation of the essay and signature of the Informed Consent form.',
  'ASC-US or LSIL or AG-US cytological result (maximum 3 months prior to inclusion) with concordant colposcopic image (performed at the screening visit).',
  'High risk HPV positive by PCR performed at screening visit.',
  'Is able, at the discretion of the researcher, to comply with the requirements of the trial and without hindrance to follow the instructions and assessments throughout it.'],
 'ExclusionCriteria': ['Clinically relevant immune system alterations, or any other autoimmune disease or in treatment with immunosuppressants.',
  'Non-diagnosed abnormal genital hemorrhage (during the 6 months prior to the screening visit).',
  'To had been vaccinated against HPV.',
  'Other symptom

In [152]:
from lxml import etree

In [153]:


paloma2["ageOfPatients"] = [str(dom.xpath('//*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[9]/td[2]/span/text()')[0])
       .strip('\n').strip('\r').strip().replace('\n','').replace('\xa0',''),
                           str(dom.xpath('//*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[9]/td[3]/span/text()')[0])
       .strip('\n').strip('\r').strip().replace('\n','').replace('\xa0','')]
paloma2["PFS"] = [dom.xpath('//*[@id="EXPAND-outcome-data-1"]/table/tbody/tr[6]/td[1]/div[1]/text()')[0]
       .strip('\n').strip('\r').strip().replace('\n','').replace('\xa0',''),
                  dom.xpath('//*[@id="EXPAND-outcome-data-1"]/table/tbody/tr[6]/td[2]/div[1]/text()')[0].strip('\n').strip().strip('\n').strip()]

paloma2["outcome_measurements"] = [dom.xpath('//*[@id="EXPAND-outcome-data-1"]/table/tbody/tr[5]/th/text()')[0]
       .strip('\n').strip('\r').strip().replace('\n','').replace('\xa0',''),
                             dom.xpath('//*[@id="EXPAND-outcome-data-1"]/table/tbody/tr[5]/th/div/text()')[0]
       .strip('\n').strip('\r').strip().replace('\n','').replace('\xa0','')]
paloma2["age_measurements"] = [dom.xpath('//*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[7]/th[1]/div/div[1]/text()')[0]
       .strip('\n').strip('\r').strip().replace('\n','').replace('\xa0',''),
                             dom.xpath('//*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[7]/th[1]/div/div[2]/text()')[0]
       .strip('\n').strip('\r').strip().replace('\n','').replace('\xa0','')]

In [154]:
a = dom.xpath('//*[@id="EXPAND-outcome-data-1"]/table/tbody/tr[6]/td[2]/div[1]/text()')[0].strip('\n').strip().strip('\n').strip()
b = dom.xpath('//*[@id="EXPAND-outcome-data-1"]/table/tbody/tr[6]/td[1]/div[1]/text()')[0].strip('\n').strip().strip('\n').strip()
b

'24.8'

In [155]:
paloma2

{'StudyType': 'Interventional',
 'InclusionCriteria': ['Woman between the ages of 30 and 65 (both inclusive).',
  'Able to read and understand the Patient Information Sheet and the Informed Consent form.',
  'Acceptance in the participation of the essay and signature of the Informed Consent form.',
  'ASC-US or LSIL or AG-US cytological result (maximum 3 months prior to inclusion) with concordant colposcopic image (performed at the screening visit).',
  'High risk HPV positive by PCR performed at screening visit.',
  'Is able, at the discretion of the researcher, to comply with the requirements of the trial and without hindrance to follow the instructions and assessments throughout it.'],
 'ExclusionCriteria': ['Clinically relevant immune system alterations, or any other autoimmune disease or in treatment with immunosuppressants.',
  'Non-diagnosed abnormal genital hemorrhage (during the 6 months prior to the screening visit).',
  'To had been vaccinated against HPV.',
  'Other symptom

In [156]:
url = "https://clinicaltrials.gov/ct2/show/results/NCT01740427?view=results"
html = urlopen(url)
soup1 = BS(html, 'html.parser')
treatment_number = soup1.find_all("td", class_="de-numValue_popFlowCell")[0].text.strip("\r").strip("\n").strip()
control_number = soup1.find_all("td", class_="de-numValue_popFlowCell")[1].text.strip("\r").strip("\n").strip()
treatment_pfs = soup1.find_all("td", class_="de-numValue_outcomeDataCell")[2].text.strip('\n').strip().split(" ")[0]
control_pfs = soup1.find_all("td", class_="de-numValue_outcomeDataCell")[3].text.strip('\n').strip().split(" ")[0]
treatment_plan = soup1.find_all("td",class_="de-outcomeLabelCell")[7].text.strip('\n').strip('\r').strip()
control_plan = soup1.find_all("td",class_="de-outcomeLabelCell")[8].text.strip('\n').strip('\r').strip()
treatment_age = soup1.find_all("td",class_="de-numValue_baselineDataCell")[11].text.strip('\n').strip('\r').strip().split(" ")[0]
control_age = soup1.find_all("td",class_="de-numValue_baselineDataCell")[12].text.strip('\n').strip('\r').strip().split(" ")[0]
hazard = soup1.find_all("td",class_="de-outcomeDataCell")[15].text.strip('\n').strip('\r').strip()

In [157]:
paloma2["numberOfPatients"] = [treatment_number,control_number]
paloma2["outcome"] = [treatment_pfs,control_pfs]
paloma2["treatment"] = [treatment_plan,control_plan]
paloma2["ageOfPatients"] = [treatment_age, control_age]
paloma2['hazardRatio'] = hazard

In [158]:
kg = Graph()
kg.bind('Clinical_Trials', CT)
kg.bind('schema',SCHEMA)
ct_uri = URIRef('https://clinicaltrials.gov/ct2/show/NCT01740427')
kg.add((ct_uri, RDF.type, SCHEMA.MedicalTrial))
kg.add((ct_uri, CT.hasID, Literal('NCT01740427')))
for i in paloma2:
    kg.add((ct_uri, CT.hasStudyType, Literal(paloma2['StudyType'])))
    if i == 'InclusionCriteria':
        for j in paloma2[i]:
            kg.add((ct_uri, CT.hasInclusionCriteria, Literal(j)))
    if i == 'ExclusionCriteria':
        for j in paloma2[i]:
            kg.add((ct_uri, CT.hasExclusionCriteria, Literal(j)))
    if i == 'Drugs':
        for j in paloma2[i]:
            k = URIRef(paloma2[i][j]['url'])
            kg.add((ct_uri, CT.hasDrug, k))
            kg.add((k, SCHEMA.name, Literal(j)))
            kg.add((k, SCHEMA.description, Literal(paloma2[i][j]['description'])))
    
    
    Group0 = paloma2["groups"][0].replace(' ',"_") + "_Group"
    Group1 = paloma2["groups"][1].replace(' ',"_") + "_Group"
    
    PFS_field = 'NCT04199078_PFS'
    kg.add((ct_uri, CT.hasPFS,CT[PFS_field]))
    kg.add((CT[PFS_field], CT.hasPFS,CT[PFS]))
    kg.add((CT[PFS], CT.hasGroup, Literal(Group0)))
    kg.add((CT[PFS], CT.hasType, Literal(paloma2["outcome_measurements"][0])))
    kg.add((CT[PFS], CT.hasUnit, Literal(paloma2["outcome_measurements"][1])))
    kg.add((CT[PFS], SCHEMA.number, Literal(paloma2['PFS'][0])))
    kg.add((CT[PFS_field], CT.hasPFS,CT[PFS1]))
    kg.add((CT[PFS1], CT.hasGroup, Literal(PFS1)))
    kg.add((CT[PFS1], CT.hasType, Literal(paloma2["outcome_measurements"][0])))
    kg.add((CT[PFS1], CT.hasUnit, Literal(paloma2["outcome_measurements"][1])))
    kg.add((CT[PFS1], SCHEMA.number, Literal(paloma2['PFS'][1])))
    
    treatment_field = 'NCT04199078_treamentDescription'
    treatment = Group0+"_Description"
    treatment1 = Group1+"_Description"
    kg.add((ct_uri, CT.hasTreatmentPlan,CT[treatment_field]))
    kg.add((CT[treatment_field], CT.hasTreatment,CT[treatment]))
    kg.add((CT[treatment], CT.hasGroup, Literal(Group0)))
    kg.add((CT[treatment], CT.hasdescription, Literal(paloma2["treatment"][0])))
    kg.add((CT[treatment_field], CT.hasTreatment,CT[treatment1]))
    kg.add((CT[treatment1], CT.hasGroup, Literal(Group1)))
    kg.add((CT[treatment1], CT.hasdescription, Literal(paloma2["treatment"][1])))
    
    nofP_field = 'NCT04199078_numberofPatients'
    numberofPatients = 'NCT04199078_numberofPatients_'+Group0
    numberofPatients1 = 'NCT04199078_numberofPatients_'+Group1
    kg.add((ct_uri, CT.hasNumberofPatients,CT[nofP_field]))
    kg.add((CT[nofP_field], CT.hasNumberofPatients,CT[numberofPatients]))
    kg.add((CT[numberofPatients], CT.hasGroup, Literal(Group0)))
    kg.add((CT[numberofPatients], SCHEMA.number, Literal(paloma2["numberOfPatients"][0])))
    kg.add((CT[numberofPatients], CT.hasUnit, Literal("person")))
    kg.add((CT[nofP_field], CT.hasNumberofPatients,CT[numberofPatients1]))
    kg.add((CT[numberofPatients1], CT.hasGroup, Literal(Group1)))
    kg.add((CT[numberofPatients1], SCHEMA.number, Literal(paloma2["numberOfPatients"][1])))
    kg.add((CT[numberofPatients1], CT.hasUnit, Literal("person")))
    
    aofP_field = 'NCT04199078_ageOfPatients'
    ageOfPatients = 'NCT04199078_ageOfPatients_'+Group0
    ageOfPatients1 = 'NCT04199078_ageOfPatients_'+Group1
    kg.add((ct_uri, CT.hasAge,CT[aofP_field]))
    kg.add((CT[aofP_field], CT.hasAgeofPatients,CT[ageOfPatients]))
    kg.add((CT[ageOfPatients], CT.hasGroup, Literal(Group0)))
    kg.add((CT[ageOfPatients], CT.hasType, Literal(paloma2["age_measurements"][0])))
    kg.add((CT[ageOfPatients], SCHEMA.number, Literal(paloma2["ageOfPatients"][0])))
    kg.add((CT[ageOfPatients], CT.hasUnit, Literal(paloma2["age_measurements"][1])))
    kg.add((CT[aofP_field], CT.hasAgeofPatients,CT[ageOfPatients1]))
    kg.add((CT[ageOfPatients1], CT.hasGroup, Literal(Group1)))
    kg.add((CT[ageOfPatients1], CT.hasType, Literal(paloma2["age_measurements"][0])))
    kg.add((CT[ageOfPatients1], SCHEMA.number, Literal(paloma2["ageOfPatients"][1])))
    kg.add((CT[ageOfPatients1], CT.hasUnit, Literal(paloma2["age_measurements"][1])))

#      'hazardRatio': '0.576',
#  'hasPercentageOfPatientsWithVisceralDisease': [48.2, 49.5],
#  'hasPercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy': [48, 49.1],
#  'hasPercentageOfPatientsWithNewlyDiagonosedAdavancedDisease': 37.2}
    PVD_field = 'NCT04199078_percentageofPatientWithVisceraDisease'
    PVD = 'NCT04199078_percentageofPatientWithVisceraDisease_'+Group0
    PVD1 = 'NCT04199078_percentageofPatientWithVisceraDisease_'+Group1
    kg.add((ct_uri, CT.hasPercentageOfPatientsWithVisceralDisease,CT[PVD_field]))
    kg.add((CT[PVD_field], CT.hasPercentageOfPatientsWithVisceralDisease,CT[PVD]))
    kg.add((CT[PVD], CT.hasGroup, Literal(Group0)))
    kg.add((CT[PVD], SCHEMA.number, Literal(paloma2["hasPercentageOfPatientsWithVisceralDisease"][0])))
    kg.add((CT[PVD], CT.hasUnit, Literal("percentage")))
    kg.add((CT[PVD_field], CT.hasPercentageOfPatientsWithVisceralDisease,CT[PVD1]))
    kg.add((CT[PVD1], CT.hasGroup, Literal(Group1)))
    kg.add((CT[PVD1], SCHEMA.number, Literal(paloma2["hasPercentageOfPatientsWithVisceralDisease"][1])))
    kg.add((CT[PVD1], CT.hasUnit, Literal("percentage")))
    
    PriorChemo_field = 'NCT04199078_percentageofPatientsHadReceivedPriorAdjuvantChemotherapy'
    PercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy = 'NCT04199078_percentageofPatientsHadReceivedPriorAdjuvantChemotherapy_'+Group0
    PercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy1 = 'NCT04199078_percentageofPatientsHadReceivedPriorAdjuvantChemotherapy_'+Group1
    kg.add((ct_uri, CT.hasPercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy,CT[PriorChemo_field]))
    kg.add((CT[PriorChemo_field], CT.hasPercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy,
            CT[PercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy]))
    kg.add((CT[PercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy], CT.hasGroup, Literal(Group0)))
    kg.add((CT[PercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy], SCHEMA.number, Literal(paloma2["hasPercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy"][0])))
    kg.add((CT[PercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy], CT.hasUnit, Literal("percentage")))
    kg.add((CT[PriorChemo_field], CT.hasPercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy,
            CT[PercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy1]))
    kg.add((CT[PercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy1], CT.hasGroup, Literal(Group1)))
    kg.add((CT[PercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy1], SCHEMA.number, Literal(paloma2["hasPercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy"][1])))
    kg.add((CT[PercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy1], CT.hasUnit, Literal("percentage")))    
    
    hazardRatio = paloma2["hazardRatio"]
    kg.add((ct_uri, CT.hasHazardRatio, CT['NCT04199078_hazardRatio']))
    kg.add((CT['NCT04199078_hazardRatio'], SCHEMA.number, Literal(paloma2["hazardRatio"])))
    kg.add((CT['NCT04199078_hazardRatio'], CT.hasConfidenceInterval, Literal("(2 sided)95% 0.463 to 0.718")))
    
    PercentageOfPatientsWithNewlyDiagonosedAdavancedDisease = str(paloma2["hasPercentageOfPatientsWithNewlyDiagonosedAdavancedDisease"])
    kg.add((ct_uri, CT.hasPercentageOfPatientsWithNewlyDiagonosedAdavancedDisease, CT['NCT04199078_PercentageOfPatientsWithNewlyDiagonosedAdavancedDisease']))
    kg.add((CT['NCT04199078_PercentageOfPatientsWithNewlyDiagonosedAdavancedDisease'], SCHEMA.number, Literal(paloma2["hasPercentageOfPatientsWithNewlyDiagonosedAdavancedDisease"])))
    kg.add((CT['NCT04199078_PercentageOfPatientsWithNewlyDiagonosedAdavancedDisease'], CT.hasUnit, Literal("percentage")))
    
kg.serialize('paloma-2.ttl', format="turtle")    

In [77]:
a

'14.7'

In [26]:
# kg.add((CT[drug], SCHEMA.Drug, SCHEMA.MedicalTrial))

## Generate knowledge graph

In [28]:
g1 = Graph()
g1.parse("CT_model.ttl", format="turtle")

g2 = Graph()
g2.parse("paloma-2.ttl", format="turtle")

final_graph = g1 + g2

final_graph.serialize("CT_paloma2.ttl", format="turtle")

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/GoogleDrive/.shortcut-targets-by-id/1O3L9NCuwRqD_wt3LT7Bcx3KjrrPL8CHI/AI Methods for Clinical Trial Comparison 2020/clinical_trials_data-structure/Knowledge_Graph_update_6:30/CT_model.ttl'

## Monaleesa2

In [80]:
monaleesa2 = dict()
studytype_url = 'https://clinicaltrials.gov/api/query/field_values?expr=monaleesa2&field=StudyType&fmt=json'
studytype = requests.get(studytype_url).json()['FieldValuesResponse']['FieldValues'][0]['FieldValue']
monaleesa2['StudyType'] = studytype
criteria_url = 'https://clinicaltrials.gov/api/query/full_studies?expr=monaleesa2&min_rnk=1&max_rnk=&fmt=json'
criteria = requests.get(criteria_url).json()['FullStudiesResponse']['FullStudies'][0]['Study']['ProtocolSection']['EligibilityModule']['EligibilityCriteria']


In [81]:
b = criteria.split('\n')
inclusion = []
exclusion = []
for i in range(0,len(b)):
    if b[i] == 'Inclusion Criteria:':
        for j in range(i+1,len(b)):
            if b[j] == 'Exclusion Criteria:':
                break
            if b[j] != '':
                inclusion.append(b[j])
    if b[i] == 'Exclusion Criteria:':
        for j in range(i+1,len(b)):
            if b[j] != '':
                exclusion.append(b[j])
monaleesa2['InclusionCriteria'] = inclusion
monaleesa2['ExclusionCriteria'] = exclusion


In [82]:
drug_url = 'https://clinicaltrials.gov/api/query/study_fields?expr=monaleesa-2&fields=ArmGroupInterventionName&min_rnk=1&max_rnk=&fmt=json'
drug = requests.get(drug_url).json()['StudyFieldsResponse']['StudyFields'][0]['ArmGroupInterventionName']
drug
drugs = [i.strip("Drug:").strip("'").strip() for i in drug] 
drugs2 = set()
for i in drugs:
    for j in i.split(" "):
        drugs2.add(j)
# drugs = list(dict.fromkeys(drugs))
drugs2

{'LEE011', 'Letrozole', 'Placebo'}

In [84]:
API_ENDPOINT = 'https://en.wikipedia.org/w/api.php'
drugs_with_des = {}
for i in drugs2:
    drugs_with_des[i] = {}
    query = i
    params = {
        'action':'query',
        'list':'search',
        'srsearch':query,
        'format':'json',
    }
    drugs_with_des[i]['url'] = requests.get(API_ENDPOINT,params=params).url
    drugs_with_des[i]['description'] = requests.get(API_ENDPOINT,params=params).json()\
    ['query']['search'][0]['snippet']
drugs_with_des

{'Placebo': {'url': 'https://en.wikipedia.org/w/api.php?action=query&list=search&srsearch=Placebo&format=json',
  'description': 'A <span class="searchmatch">placebo</span> (/pləˈsiːboʊ/ plə-SEE-boh) is a substance or treatment which is designed to have no therapeutic value. Common <span class="searchmatch">placebos</span> include inert tablets'},
 'LEE011': {'url': 'https://en.wikipedia.org/w/api.php?action=query&list=search&srsearch=LEE011&format=json',
  'description': 'Epithelioid sarcoma is a rare soft tissue sarcoma arising from mesenchymal tissue and characterized by epithelioid-like features. It accounts for less'},
 'Letrozole': {'url': 'https://en.wikipedia.org/w/api.php?action=query&list=search&srsearch=Letrozole&format=json',
  'description': '<span class="searchmatch">Letrozole</span>, sold under the brand name Femara among others, is an aromatase inhibitor which is used in the treatment of hormonally-responsive breast cancer'}}

In [85]:
monaleesa2['Drugs'] = drugs_with_des

In [86]:
outcome_url = "https://clinicaltrials.gov/api/query/study_fields?expr=monaleesa2&fields=OutcomeMeasurementValue&min_rnk=1&max_rnk=&fmt=json"
outcome = requests.get(outcome_url).json()["StudyFieldsResponse"]["StudyFields"][0]["OutcomeMeasurementValue"][:2]

treatment_url = "https://clinicaltrials.gov/api/query/study_fields?expr=monaleesa2&fields=ArmGroupDescription&min_rnk=1&max_rnk=&fmt=json"
treatment = requests.get(treatment_url).json()["StudyFieldsResponse"]["StudyFields"][0]["ArmGroupDescription"]

numberOfPatients_url = "https://clinicaltrials.gov/api/query/study_fields?expr=monaleesa2&fields=BaselineDenomCountValue&min_rnk=1&max_rnk=&fmt=json"
numberOfPatients = requests.get(numberOfPatients_url).json()["StudyFieldsResponse"]["StudyFields"][0]["BaselineDenomCountValue"]

ageOfPatients_url = "https://clinicaltrials.gov/api/query/study_fields?expr=monaleesa2&fields=BaselineMeasurementValue&min_rnk=1&max_rnk=&fmt=json"
ageOfPatients = requests.get(ageOfPatients_url).json()["StudyFieldsResponse"]["StudyFields"][0]["BaselineMeasurementValue"][:3]
ageOfPatients

hazardRatio_url = "https://clinicaltrials.gov/api/query/study_fields?expr=monaleesa2&fields=OutcomeAnalysisParamValue&min_rnk=1&max_rnk=&fmt=json"
hazardRatio = requests.get(hazardRatio_url).json()["StudyFieldsResponse"]["StudyFields"][0]["OutcomeAnalysisParamValue"][0]
hazardRatio

'0.556'

In [162]:
# monaleesa2["outcome"] = outcome
# monaleesa2["treatment"] = treatment
# monaleesa2["numberOfPatients"] = numberOfPatients
# monaleesa2["ageOfPatients"] = ageOfPatients
# monaleesa2["hazardRatio"] = hazardRatio
url = "https://clinicaltrials.gov/ct2/show/results/NCT01958021?view=results"
html = urlopen(url)
soup = BS(html, 'html.parser')

monaleesa2["numberOfPatients"] = []
monaleesa2["numberOfPatients"].append(soup.find_all("td", class_="de-numValue_popFlowCell")[0].text.strip("\r").strip("\n").strip())
monaleesa2["numberOfPatients"].append(soup.find_all("td", class_="de-numValue_popFlowCell")[1].text.strip("\r").strip("\n").strip())
monaleesa2["outcome"] = []
monaleesa2["outcome"].append(soup.find_all("td", class_="de-numValue_outcomeDataCell")[2].text.strip('\n').strip().split(" ")[0])
monaleesa2["outcome"].append(soup.find_all("td", class_="de-numValue_outcomeDataCell")[3].text.strip('\n').strip().split(" ")[0])
monaleesa2["treatment"] = []
monaleesa2["treatment"].append(soup.find_all("td",class_="de-outcomeLabelCell")[7].text.strip('\n').strip('\r').strip())
monaleesa2["treatment"].append(soup.find_all("td",class_="de-outcomeLabelCell")[8].text.strip('\n').strip('\r').strip())

monaleesa2["hazardRatio"] = soup.find_all("td",class_="de-outcomeDataCell")[15].text.strip('\n').strip('\r').strip()
monaleesa2["hasPercentageOfPatientsWithVisceralDisease"] = 77.5
monaleesa2["hasPercentageOfPatientsWithVisceralDisease"] = [48.2,49.5]
monaleesa2["hasPercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy"] = [48,49.1]
monaleesa2["hasPercentageOfPatientsWithNewlyDiagonosedAdavancedDisease"] = 37.2
url = "https://clinicaltrials.gov/ct2/show/results/NCT01958021?view=results"
response = urlopen(url)
soup = BS(response, 'html.parser')
baseline = soup.find_all("th",class_="de-popFlowLabelCell")
groups = []
for i in range(0,len(baseline)):
    groups.append(baseline[i].text.strip("\n").strip("\r").strip())
    number_of_patients.append(soup.find_all("td",class_="de-numValue_baselineDataCell")[i].text.strip("\n").strip("\r").strip())
monaleesa2["groups"] = groups
dom = etree.HTML(str(soup))
monaleesa2['PFS'] = [dom.xpath('//*[@id="EXPAND-outcome-data-1"]/table/tbody/tr[6]/td[1]/div[1]/text()')[0]
       .strip('\n').strip('\r').strip().replace('\n','').replace('\xa0',''),
                  dom.xpath('//*[@id="EXPAND-outcome-data-1"]/table/tbody/tr[6]/td[2]/div[1]/text()')[0].strip('\n').strip().strip('\n').strip()]
monaleesa2["outcome_measurements"] = [dom.xpath('//*[@id="EXPAND-outcome-data-1"]/table/tbody/tr[5]/th/text()')[0]
       .strip('\n').strip('\r').strip().replace('\n','').replace('\xa0',''),
                             dom.xpath('//*[@id="EXPAND-outcome-data-1"]/table/tbody/tr[5]/th/div/text()')[0]
       .strip('\n').strip('\r').strip().replace('\n','').replace('\xa0','')]
monaleesa2["age_measurements"] = [dom.xpath('//*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[7]/th[1]/div/div[1]/text()')[0]
       .strip('\n').strip('\r').strip().replace('\n','').replace('\xa0',''),
                             dom.xpath('//*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[7]/th[1]/div/div[2]/text()')[0]
       .strip('\n').strip('\r').strip().replace('\n','').replace('\xa0','')]
monaleesa2["ageOfPatients"] = [dom.xpath('//*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[9]/td[2]/span/text()')[0]
       .strip('\n').strip('\r').strip().replace('\n','').replace('\xa0',''),
                               dom.xpath('//*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[9]/td[3]/span/text()')[0]
       .strip('\n').strip('\r').strip().replace('\n','').replace('\xa0','')]

monaleesa2

{'StudyType': 'Interventional',
 'InclusionCriteria': ['Women with advanced (locoregionally recurrent or metastatic) breast cancer not amenable to curative therapy.',
  'Patient is postmenopausal. Postmenopausal status is defined either by:',
  'Prior bilateral oophorectomy',
  'Age ≥60',
  'Age <60 and amenorrhea for 12 or more months (in the absence of chemotherapy, tamoxifen, toremifen, or ovarian suppression) and FSH and estradiol in the postmenopausal range per local normal range Note: For women with therapy-induced amenorrhea, serial measurements of FSH and/or estradiol are needed to ensure postmenopausal status. Ovarian radiation or treatment with a luteinizing hormone-releasing hormone agonist (LH-RHa) (goserelin acetate or leuprolide acetate) is not permitted for induction of ovarian suppression in this trial.',
  'No prior systemic anti-cancer therapy for advanced disease.',
  'Patient has a histologically and/or cytologically confirmed diagnosis of estrogen-receptor positive

In [120]:

kg = Graph()
kg.bind('Clinical_Trials', CT)
kg.bind('schema',SCHEMA)
ct_uri = URIRef('https://clinicaltrials.gov/ct2/show/NCT01958021')
kg.add((ct_uri, RDF.type, SCHEMA.MedicalTrial))
kg.add((ct_uri, CT.hasID, Literal('NCT01958021')))
for i in monaleesa2:
    kg.add((ct_uri, CT.hasStudyType, Literal(monaleesa2['StudyType'])))
    if i == 'InclusionCriteria':
        for j in monaleesa2[i]:
            kg.add((ct_uri, CT.hasInclusionCriteria, Literal(j)))
    if i == 'ExclusionCriteria':
        for j in monaleesa2[i]:
            kg.add((ct_uri, CT.hasExclusionCriteria, Literal(j)))
    if i == 'Drugs':
        for j in monaleesa2[i]:
            k = URIRef(monaleesa2[i][j]['url'])
            kg.add((ct_uri, CT.hasDrug, k))
            kg.add((k, SCHEMA.name, Literal(j)))
            kg.add((k, SCHEMA.description, Literal(monaleesa2[i][j]['description'])))
    
    
    Group = monaleesa2["groups"][0].replace(' ',"_") + "_Group"
    Group1 = monaleesa2["groups"][1].replace(' ',"_") + "_Group"
    
    PFS_field = 'NCT01958021_PFS'
    kg.add((ct_uri, CT.hasPFS,CT[PFS_field]))
    kg.add((CT[PFS_field], CT.hasPFS,CT[PFS]))
    kg.add((CT[PFS], CT.hasGroup, Literal(Group)))
    kg.add((CT[PFS], CT.hasType, Literal(monaleesa2["outcome_measurements"][0])))
    kg.add((CT[PFS], CT.hasUnit, Literal(monaleesa2["outcome_measurements"][1])))
    kg.add((CT[PFS], SCHEMA.number, Literal(monaleesa2['PFS'][0])))
    kg.add((CT[PFS_field], CT.hasPFS,CT[PFS1]))
    kg.add((CT[PFS1], CT.hasGroup, Literal(Group1)))
    kg.add((CT[PFS1], CT.hasType, Literal(monaleesa2["outcome_measurements"][0])))
    kg.add((CT[PFS1], CT.hasUnit, Literal(monaleesa2["outcome_measurements"][1])))
    kg.add((CT[PFS1], SCHEMA.number, Literal(monaleesa2['PFS'][1])))
    
    treatment_field = 'NCT01958021_treamentDescription'
    treatment = Group+"_Description"
    treatment1 = Group1+"_Description"
    kg.add((ct_uri, CT.hasTreatmentPlan,CT[treatment_field]))
    kg.add((CT[treatment_field], CT.hasTreatment,CT[treatment]))
    kg.add((CT[treatment], CT.hasGroup, Literal(Group)))
    kg.add((CT[treatment], CT.hasdescription, Literal(monaleesa2["treatment"][0])))
    kg.add((CT[treatment_field], CT.hasTreatment,CT[treatment1]))
    kg.add((CT[treatment1], CT.hasGroup, Literal(Group1)))
    kg.add((CT[treatment1], CT.hasdescription, Literal(monaleesa2["treatment"][1])))
    
    nofP_field = 'NCT01958021_numberofPatients'
    numberofPatients = 'NCT01958021_numberofPatients_'+Group
    numberofPatients1 = 'NCT01958021_numberofPatients_'+Group1
    kg.add((ct_uri, CT.hasNumberofPatients,CT[nofP_field]))
    kg.add((CT[nofP_field], CT.hasNumberofPatients,CT[numberofPatients]))
    kg.add((CT[numberofPatients], CT.hasGroup, Literal(Group)))
    kg.add((CT[numberofPatients], SCHEMA.number, Literal(monaleesa2["numberOfPatients"][0])))
    kg.add((CT[numberofPatients], CT.hasUnit, Literal("person")))
    kg.add((CT[nofP_field], CT.hasNumberofPatients,CT[numberofPatients1]))
    kg.add((CT[numberofPatients1], CT.hasGroup, Literal(Group1)))
    kg.add((CT[numberofPatients1], SCHEMA.number, Literal(monaleesa2["numberOfPatients"][1])))
    kg.add((CT[numberofPatients1], CT.hasUnit, Literal("person")))
    
    aofP_field = 'NCT01958021_ageOfPatients'
    ageOfPatients = 'NCT01958021_ageOfPatients_'+Group
    ageOfPatients1 = 'NCT01958021_ageOfPatients_'+Group1
    kg.add((ct_uri, CT.hasAge,CT[aofP_field]))
    kg.add((CT[aofP_field], CT.hasAgeofPatients,CT[ageOfPatients]))
    kg.add((CT[ageOfPatients], CT.hasGroup, Literal(Group)))
    kg.add((CT[ageOfPatients], CT.hasType, Literal(monaleesa2["age_measurements"][0])))
    kg.add((CT[ageOfPatients], SCHEMA.number, Literal(monaleesa2["ageOfPatients"][0])))
    kg.add((CT[ageOfPatients], CT.hasUnit, Literal(monaleesa2["age_measurements"][1])))
    kg.add((CT[aofP_field], CT.hasAgeofPatients,CT[ageOfPatients1]))
    kg.add((CT[ageOfPatients1], CT.hasGroup, Literal(Group1)))
    kg.add((CT[ageOfPatients1], CT.hasType, Literal(monaleesa2["age_measurements"][0])))
    kg.add((CT[ageOfPatients1], SCHEMA.number, Literal(monaleesa2["ageOfPatients"][1])))
    kg.add((CT[ageOfPatients1], CT.hasUnit, Literal(monaleesa2["age_measurements"][1])))

#      'hazardRatio': '0.576',
#  'hasPercentageOfPatientsWithVisceralDisease': [48.2, 49.5],
#  'hasPercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy': [48, 49.1],
#  'hasPercentageOfPatientsWithNewlyDiagonosedAdavancedDisease': 37.2}
    PVD_field = 'NCT01958021_percentageofPatientWithVisceraDisease'
    PVD = 'NCT01958021_percentageofPatientWithVisceraDisease_'+Group
    PVD1 = 'NCT01958021_percentageofPatientWithVisceraDisease_'+Group1
    kg.add((ct_uri, CT.hasPercentageOfPatientsWithVisceralDisease,CT[PVD_field]))
    kg.add((CT[PVD_field], CT.hasPercentageOfPatientsWithVisceralDisease,CT[PVD]))
    kg.add((CT[PVD], CT.hasGroup, Literal(Group)))
    kg.add((CT[PVD], SCHEMA.number, Literal(monaleesa2["hasPercentageOfPatientsWithVisceralDisease"][0])))
    kg.add((CT[PVD], CT.hasUnit, Literal("percentage")))
    kg.add((CT[PVD_field], CT.hasPercentageOfPatientsWithVisceralDisease,CT[PVD1]))
    kg.add((CT[PVD1], CT.hasGroup, Literal(Group1)))
    kg.add((CT[PVD1], SCHEMA.number, Literal(monaleesa2["hasPercentageOfPatientsWithVisceralDisease"][1])))
    kg.add((CT[PVD1], CT.hasUnit, Literal("percentage")))
    
    PriorChemo_field = 'NCT01958021_percentageofPatientsHadReceivedPriorAdjuvantChemotherapy'
    PercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy = 'NCT01958021_percentageofPatientsHadReceivedPriorAdjuvantChemotherapy_'+Group
    PercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy1 = 'NCT01958021_percentageofPatientsHadReceivedPriorAdjuvantChemotherapy_'+Group1
    kg.add((ct_uri, CT.hasPercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy,CT[PriorChemo_field]))
    kg.add((CT[PriorChemo_field], CT.hasPercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy,
            CT[PercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy]))
    kg.add((CT[PercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy], CT.hasGroup, Literal(Group)))
    kg.add((CT[PercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy], SCHEMA.number, Literal(monaleesa2["hasPercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy"][0])))
    kg.add((CT[PercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy], CT.hasUnit, Literal("percentage")))
    kg.add((CT[PriorChemo_field], CT.hasPercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy,
            CT[PercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy1]))
    kg.add((CT[PercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy1], CT.hasGroup, Literal(Group1)))
    kg.add((CT[PercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy1], SCHEMA.number, Literal(monaleesa2["hasPercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy"][1])))
    kg.add((CT[PercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy1], CT.hasUnit, Literal("percentage")))    
    
    hazardRatio = monaleesa2["hazardRatio"]
    kg.add((ct_uri, CT.hasHazardRatio, CT['NCT01958021_hazardRatio']))
    kg.add((CT['NCT01958021_hazardRatio'], SCHEMA.number, Literal(monaleesa2["hazardRatio"])))
    kg.add((CT['NCT01958021_hazardRatio'], CT.hasConfidenceInterval, Literal("(2 sided)95% 0.463 to 0.718")))
    
    PercentageOfPatientsWithNewlyDiagonosedAdavancedDisease = str(monaleesa2["hasPercentageOfPatientsWithNewlyDiagonosedAdavancedDisease"])
    kg.add((ct_uri, CT.hasPercentageOfPatientsWithNewlyDiagonosedAdavancedDisease, CT['NCT01958021_PercentageOfPatientsWithNewlyDiagonosedAdavancedDisease']))
    kg.add((CT['NCT01958021_PercentageOfPatientsWithNewlyDiagonosedAdavancedDisease'], SCHEMA.number, Literal(monaleesa2["hasPercentageOfPatientsWithNewlyDiagonosedAdavancedDisease"])))
    kg.add((CT['NCT01958021_PercentageOfPatientsWithNewlyDiagonosedAdavancedDisease'], CT.hasUnit, Literal("percentage")))
    
kg.serialize('monaleesa-2.ttl', format="turtle")     
    

In [74]:
kg = Graph()
kg.bind('Clinical_Trials', CT)
kg.bind('schema',SCHEMA)
ct_uri = URIRef('https://clinicaltrials.gov/ct2/show/NCT00984399')
kg.add((ct_uri, CT['hasNoResults'], Literal("No Results")))
kg.serialize('new.ttl', format="turtle")  
g1 = Graph()
g1.parse("NCT00984399.ttl", format="turtle")

g2 = Graph()
g2.parse("new.ttl", format="turtle")

final_graph = g1 + g2

final_graph.serialize("new_NCT00984399.ttl", format="turtle")


In [75]:
kg = Graph()
kg.bind('Clinical_Trials', CT)
kg.bind('schema',SCHEMA)
ct_uri = URIRef('https://clinicaltrials.gov/ct2/show/NCT01478477')
kg.add((ct_uri, CT['hasNoResults'], Literal("No Results")))
kg.serialize('new.ttl', format="turtle")  
g1 = Graph()
g1.parse("NCT01478477.ttl", format="turtle")

g2 = Graph()
g2.parse("new.ttl", format="turtle")

final_graph = g1 + g2

final_graph.serialize("new_NCT01478477.ttl", format="turtle")

In [76]:
kg = Graph()
kg.bind('Clinical_Trials', CT)
kg.bind('schema',SCHEMA)
ct_uri = URIRef('https://clinicaltrials.gov/ct2/show/NCT01676753')
kg.add((ct_uri, CT['hasNoResults'], Literal("No Results")))
kg.serialize('new.ttl', format="turtle")  
g1 = Graph()
g1.parse("NCT01676753.ttl", format="turtle")

g2 = Graph()
g2.parse("new.ttl", format="turtle")

final_graph = g1 + g2

final_graph.serialize("new_graphs/new_NCT01676753.ttl", format="turtle")

In [77]:
kg = Graph()
kg.bind('Clinical_Trials', CT)
kg.bind('schema',SCHEMA)
ct_uri = URIRef('https://clinicaltrials.gov/ct2/show/NCT01791478')
kg.add((ct_uri, CT['hasNoResults'], Literal("No Results")))
kg.serialize('new.ttl', format="turtle")  
g1 = Graph()
g1.parse("NCT01791478.ttl", format="turtle")

g2 = Graph()
g2.parse("new.ttl", format="turtle")

final_graph = g1 + g2

final_graph.serialize("new_graphs/new_NCT01791478.ttl", format="turtle")

In [78]:
kg = Graph()
kg.bind('Clinical_Trials', CT)
kg.bind('schema',SCHEMA)
ct_uri = URIRef('https://clinicaltrials.gov/ct2/show/NCT01824836')
kg.add((ct_uri, CT['hasNoResults'], Literal("No Results")))
kg.serialize('new.ttl', format="turtle")  
g1 = Graph()
g1.parse("NCT01824836.ttl", format="turtle")

g2 = Graph()
g2.parse("new.ttl", format="turtle")

final_graph = g1 + g2

final_graph.serialize("new_graphs/new_NCT01824836.ttl", format="turtle")

In [81]:
kg = Graph()
kg.bind('Clinical_Trials', CT)
kg.bind('schema',SCHEMA)
ct_uri = URIRef('https://clinicaltrials.gov/ct2/show/NCT02203513')
kg.add((ct_uri, CT['hasNoResults'], Literal("No Results")))
kg.serialize('new.ttl', format="turtle")  
g1 = Graph()
g1.parse("NCT02203513.ttl", format="turtle")

g2 = Graph()
g2.parse("new.ttl", format="turtle")

final_graph = g1 + g2

final_graph.serialize("new_graphs/new_NCT02203513.ttl", format="turtle")

In [82]:
kg = Graph()
kg.bind('Clinical_Trials', CT)
kg.bind('schema',SCHEMA)
ct_uri = URIRef('https://clinicaltrials.gov/ct2/show/NCT02379247')
kg.add((ct_uri, CT['hasNoResults'], Literal("Results not posted on ClinicalTrials.gov")))
kg.serialize('new.ttl', format="turtle")  
g1 = Graph()
g1.parse("NCT02379247.ttl", format="turtle")

g2 = Graph()
g2.parse("new.ttl", format="turtle")

final_graph = g1 + g2

final_graph.serialize("new_graphs/new_NCT02379247.ttl", format="turtle")

In [83]:
kg = Graph()
kg.bind('Clinical_Trials', CT)
kg.bind('schema',SCHEMA)
ct_uri = URIRef('https://clinicaltrials.gov/ct2/show/NCT02530489')
kg.add((ct_uri, CT['hasNoResults'], Literal("No Results")))
kg.serialize('new.ttl', format="turtle")  
g1 = Graph()
g1.parse("NCT02530489.ttl", format="turtle")

g2 = Graph()
g2.parse("new.ttl", format="turtle")

final_graph = g1 + g2

final_graph.serialize("new_graphs/new_NCT02530489.ttl", format="turtle")

In [84]:
kg = Graph()
kg.bind('Clinical_Trials', CT)
kg.bind('schema',SCHEMA)
ct_uri = URIRef('https://clinicaltrials.gov/ct2/show/NCT02657343')
kg.add((ct_uri, CT['hasNoResults'], Literal("No Results")))
kg.serialize('new.ttl', format="turtle")  
g1 = Graph()
g1.parse("NCT02657343.ttl", format="turtle")

g2 = Graph()
g2.parse("new.ttl", format="turtle")

final_graph = g1 + g2

final_graph.serialize("new_graphs/new_NCT02657343.ttl", format="turtle")

In [85]:
kg = Graph()
kg.bind('Clinical_Trials', CT)
kg.bind('schema',SCHEMA)
ct_uri = URIRef('https://clinicaltrials.gov/ct2/show/NCT02672475')
kg.add((ct_uri, CT['hasNoResults'], Literal("No Results")))
kg.serialize('new.ttl', format="turtle")  
g1 = Graph()
g1.parse("NCT02672475.ttl", format="turtle")

g2 = Graph()
g2.parse("new.ttl", format="turtle")

final_graph = g1 + g2

final_graph.serialize("new_graphs/new_NCT02672475.ttl", format="turtle")

In [86]:
kg = Graph()
kg.bind('Clinical_Trials', CT)
kg.bind('schema',SCHEMA)
ct_uri = URIRef('https://clinicaltrials.gov/ct2/show/NCT02719691')
kg.add((ct_uri, CT['hasNoResults'], Literal("No Results")))
kg.serialize('new.ttl', format="turtle")  
g1 = Graph()
g1.parse("NCT02719691.ttl", format="turtle")

g2 = Graph()
g2.parse("new.ttl", format="turtle")

final_graph = g1 + g2

final_graph.serialize("new_graphs/new_NCT02719691.ttl", format="turtle")

In [87]:
kg = Graph()
kg.bind('Clinical_Trials', CT)
kg.bind('schema',SCHEMA)
ct_uri = URIRef('https://clinicaltrials.gov/ct2/show/NCT03219476')
kg.add((ct_uri, CT['hasNoResults'], Literal("No Results")))
kg.serialize('new.ttl', format="turtle")  
g1 = Graph()
g1.parse("NCT03219476.ttl", format="turtle")

g2 = Graph()
g2.parse("new.ttl", format="turtle")

final_graph = g1 + g2

final_graph.serialize("new_graphs/new_NCT03219476.ttl", format="turtle")

In [88]:
kg = Graph()
kg.bind('Clinical_Trials', CT)
kg.bind('schema',SCHEMA)
ct_uri = URIRef('https://clinicaltrials.gov/ct2/show/NCT03874325')
kg.add((ct_uri, CT['hasNoResults'], Literal("No Results")))
kg.serialize('new.ttl', format="turtle")  
g1 = Graph()
g1.parse("NCT03874325.ttl", format="turtle")

g2 = Graph()
g2.parse("new.ttl", format="turtle")

final_graph = g1 + g2

final_graph.serialize("new_graphs/new_NCT03874325.ttl", format="turtle")

In [94]:
url = "https://clinicaltrials.gov/ct2/show/results/NCT03093350?view=results"
response = urlopen(url)
soup = BS(response, 'html.parser')
dom = etree.HTML(str(soup))
## age: td[2]
a = [str(dom.xpath('//*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[9]/td[2]/span/text()')[0])
       .strip('\n').strip('\r').strip().replace('\n','').replace('\xa0','')]
## outcome: td[1]
b = [dom.xpath('//*[@id="EXPAND-outcome-data-1"]/table/tbody/tr[6]/td[1]/div[1]/text()')[0]
       .strip('\n').strip('\r').strip().replace('\n','').replace('\xa0','')]
b

['1']

In [167]:
url = "https://clinicaltrials.gov/ct2/show/results/NCT03093350?view=results"
response = urlopen(url)
soup = BS(response, 'html.parser')
baseline = soup.find_all("th",class_="de-popFlowLabelCell")
groups = []
number_of_patients = []
for i in range(0,len(baseline)):
    groups.append(baseline[i].text.strip("\n").strip("\r").strip())
    number_of_patients.append(soup.find_all("td",class_="de-numValue_baselineDataCell")[i].text.strip("\n").strip("\r").strip())

response = urlopen(url)
soup = BS(response, 'html.parser')
dom = etree.HTML(str(soup))

NCT03093350 = dict()
NCT03093350["numberOfPatients"] = []
NCT03093350["numberOfPatients"].append(soup.find_all("td", class_="de-numValue_popFlowCell")[0].text.strip("\r").strip("\n").strip())
NCT03093350["numberOfPatients"].append(soup.find_all("td", class_="de-numValue_popFlowCell")[1].text.strip("\r").strip("\n").strip())
NCT03093350["treatment"] = dom.xpath('//*[@id="EXPAND-armGroupDescriptionRow-baseline"]/td[2]/text()')
NCT03093350["ageOfPatients"] = [dom.xpath('//*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[9]/td[2]/span/text()')[0]
       .strip('\n').strip('\r').strip().replace('\n','').replace('\xa0','')]
# NCT03093350["hazardRatio"] = soup.find_all("td",class_="de-outcomeDataCell")[15].text.strip('\n').strip('\r').strip()

NCT03093350['outcome'] = [dom.xpath('//*[@id="EXPAND-outcome-data-1"]/table/tbody/tr[6]/td[1]/div[1]/text()')[0]
       .strip('\n').strip('\r').strip().replace('\n','').replace('\xa0','')]
NCT03093350["outcome_measurements"] = [dom.xpath('//*[@id="EXPAND-outcome-data-1"]/table/tbody/tr[5]/th/text()')[0]
       .strip('\n').strip('\r').strip().replace('\n','').replace('\xa0',''),
                             dom.xpath('//*[@id="EXPAND-outcome-data-1"]/table/tbody/tr[5]/th/div/text()')[0]
       .strip('\n').strip('\r').strip().replace('\n','').replace('\xa0','')]
NCT03093350["age_measurements"] = [dom.xpath('//*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[7]/th[1]/div/div[1]/text()')[0]
       .strip('\n').strip('\r').strip().replace('\n','').replace('\xa0',''),
                             dom.xpath('//*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[7]/th[1]/div/div[2]/text()')[0]
       .strip('\n').strip('\r').strip().replace('\n','').replace('\xa0','')]
NCT03093350



{'numberOfPatients': ['12', '10'],
 'treatment': ['\n', '\n', '\n', '\n', '\n', '\n', '\n'],
 'ageOfPatients': ['58.80        (11.91)'],
 'outcome': ['1'],
 'outcome_measurements': ['Measure Type:Count of Participants',
  'Unit of Measure: Participants'],
 'age_measurements': ['Mean (Standard Deviation)', 'Unit of measure: Years']}

In [297]:
def crawling_data(ID):
    url = 'https://clinicaltrials.gov/ct2/show/results/'+ID+'?view=results'
    
    response = urlopen(url)
    soup = BS(response, 'html.parser')
    baseline = soup.find_all("th",class_="de-popFlowLabelCell")
    response = urlopen(url)
    soup = BS(response, 'html.parser')
    dom = etree.HTML(str(soup))
    output = dict()
    output['groups'] = []
    output["treatment"] = []
    output["ageOfPatients"] = []
    output["outcome"] = []
    output["outcome_measurements"] = []
    output["age_measurements"] = []
    output["outcome_description"] = []

    if (len(baseline)>0):
        groups = []
        for i in range(0,len(baseline)):
            groups.append(baseline[i].text.strip("\n").strip("\r").strip())
            number_of_patients.append(soup.find_all("td",class_="de-numValue_baselineDataCell")[i].text.strip("\n").strip("\r").strip())
        
        output['groups'] = groups
        output["numberOfPatients"] = []
        if(len(soup.find_all("td", class_="de-numValue_popFlowCell"))>0):          
            for i in range(0,len(baseline)):
                output["numberOfPatients"].append(soup.find_all("td", class_="de-numValue_popFlowCell")[i].text.strip("\r").strip("\n").strip())

 
        treatment = dom.xpath('//*[@id="EXPAND-armGroupDescriptionRow-baseline"]/td[2]/text()')
        if(len(treatment)>0):
            print('1')
            output["treatment"].append(dom.xpath('//*[@id="EXPAND-armGroupDescriptionRow-baseline"]/td[' + str(i+2)+ ']/text()')[0]
                                       .strip('\n').strip('\r').strip().replace('\n','').replace('\xa0',''))
        else:
            print('2')
            treatment = dom.xpath('//*[@id="EXPAND-armGroupDescriptionRow-baseline"]/td[2]/p[1]/text()')
            if(len(treatment)>0):
                print(treatment)
                output["treatment"].append(dom.xpath('//*[@id="EXPAND-armGroupDescriptionRow-baseline"]/td['+str(i+2)+']/p[1]/text()')[0]
                                           .strip('\n').strip('\r').strip().replace('\n','').replace('\xa0',''))
          
        output["age_measurements"] = [dom.xpath('//*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[7]/th[1]/div/div[1]/text()'),
                                     dom.xpath('//*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[7]/th[1]/div/div[2]/text()')]
        if(len(output["age_measurements"])>0):
            output["age_measurements"] = [dom.xpath('//*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[7]/th[1]/div/div[1]/text()')
                                          [0].strip('\n').strip('\r').strip().replace('\n','').replace('\xa0',''),
                                     dom.xpath('//*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[7]/th[1]/div/div[2]/text()')
                                         [0].strip('\n').strip('\r').strip().replace('\n','').replace('\xa0','')]

        if('count' in output["age_measurements"][0].lower()):
            print("need to check count")
        else:
            
            age = dom.xpath('//*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[9]/td[2]/span/text()')
    #         //*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[9]/td[2]/div[1]
            if(len(age)>0):
                for i in range(0,len(baseline)):
                    output["ageOfPatients"].append(dom.xpath('//*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[9]/td[' + str(i+2) + ']/span/text()')
                                                   [0].strip('\n').strip('\r').strip().replace('\n','').replace('\xa0',''))
            else:
                age = dom.xpath('//*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[9]/td[2]/div[1]/text()') 
                for i in range(0,len(baseline)):
                    output["ageOfPatients"].append(dom.xpath('//*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[9]/td['+str(i+2)+']/div[1]/text()')
                                                   [0].strip('\n').strip('\r').strip().replace('\n','').replace('\xa0',''))
    # //*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[9]/td[2]/div[2]
    # //*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[10]/td[2]/div[1]

        outcome = dom.xpath('//*[@id="EXPAND-outcome-data-1"]/table/tbody/tr[6]/td[1]/div[1]/text()')
        
        if(len(outcome) > 0):
            for i in range(0,len(baseline)):
                output["outcome"].append(dom.xpath('//*[@id="EXPAND-outcome-data-1"]/table/tbody/tr[6]/td[' + str(i+1) + ']/div[1]/text()')
                                         [0].strip('\n').strip('\r').strip().replace('\n','').replace('\xa0',''))
        else:
            output["outcome"] = len(outcome)
        output["outcome_measurements"] = [dom.xpath('//*[@id="EXPAND-outcome-data-1"]/table/tbody/tr[5]/th/text()'),
                                     dom.xpath('//*[@id="EXPAND-outcome-data-1"]/table/tbody/tr[5]/th/div/text()')]
        if(len(output["outcome_measurements"]) > 0):
            output["outcome_measurements"] = [dom.xpath('//*[@id="EXPAND-outcome-data-1"]/table/tbody/tr[5]/th/text()')
                                              [0].strip('\n').strip('\r').strip().replace('\n','').replace('\xa0',''),
                                     dom.xpath('//*[@id="EXPAND-outcome-data-1"]/table/tbody/tr[5]/th/div/text()')
                                             [0].strip('\n').strip('\r').strip().replace('\n','').replace('\xa0','')]
        outcome_description = dom.xpath('//*[@id="tab-body"]/div[1]/div[4]/div[2]/fieldset[1]/div/table/tbody/tr[1]/td/text()')
        if (len(outcome_description)>0):
            print('0')
            output["outcome_description"] = dom.xpath('//*[@id="tab-body"]/div[1]/div[4]/div[2]/fieldset[1]/div/table/tbody/tr[1]/td/text()')
            [0].strip('\n').strip('\r').strip().replace('\n','').replace('\xa0','')
    return output





In [312]:
url = 'https://clinicaltrials.gov/ct2/show/results/NCT03093350?view=results'
    
response = urlopen(url)
soup = BS(response, 'html.parser')
baseline = soup.find_all("th",class_="de-popFlowLabelCell")
response = urlopen(url)
soup = BS(response, 'html.parser')
dom = etree.HTML(str(soup))
treatment = dom.xpath('//*[@id="EXPAND-armGroupDescriptionRow-baseline"]/td[2]//text()')
description = dom.xpath('//*[@id="tab-body"]/div[1]/div[4]/div[2]/fieldset[1]/div/table/tbody/tr[1]/td/text()')
description



[]

In [299]:
print(crawling_data('NCT02028221'))


1
{'groups': ['Placebo', 'Metformin'], 'treatment': [''], 'ageOfPatients': ['39.2        (8.6)', '39.9        (7.9)'], 'outcome': 0, 'outcome_measurements': ['Mean (Standard Deviation)', 'Unit of Measure: cm^3'], 'age_measurements': ['Mean (Standard Deviation)', 'Unit of measure: Years'], 'outcome_description': [], 'numberOfPatients': ['75', '76']}


In [300]:
print(crawling_data('NCT00003042'))

1
{'groups': ['Neoadjuvant Chemo Followed by Surgery & High-dose Chemo With PSC Rescue', 'High-dose Chemo With Rescue'], 'treatment': [''], 'ageOfPatients': ['49', '50'], 'outcome': ['61.5', '77.8'], 'outcome_measurements': ['Measure Type:Number', 'Unit of Measure: percentage of participants'], 'age_measurements': ['Median (Full Range)', 'Unit of measure: Years'], 'outcome_description': [], 'numberOfPatients': ['14', '27']}


In [301]:

print(crawling_data('NCT02278120'))

1
{'groups': ['LEE011 + NSAI/Tamoxifen + Goserelin', 'LEE011 Placebo + NSAI/Tamoxifen+ Goserelin'], 'treatment': ['LEE011 Placebo 600 mg daily oral (3 weeks on/ 1 week off) in combination with NSAI or tamoxifen (tamoxifen 20 mg daily oral or letrozole 2.5 mg daily oral or anastrozole 1 mg daily oral) and goserelin 3.6 mg subcutaneous injection (once every 28 days)'], 'ageOfPatients': ['42.6        (6.6)', '43.7        (6.17)'], 'outcome': ['23.8', '13.0'], 'outcome_measurements': ['Median (95% Confidence Interval)', 'Unit of Measure: Months'], 'age_measurements': ['Mean (Standard Deviation)', 'Unit of measure: Years'], 'outcome_description': [], 'numberOfPatients': ['335', '337']}


In [302]:
print(crawling_data('NCT01035099'))

1
need to check count
{'groups': ['Titrated Dose Letrozole', 'Fixed Dose Letrozole'], 'treatment': [''], 'ageOfPatients': [], 'outcome': 0, 'outcome_measurements': ['Mean (Standard Deviation)', 'Unit of Measure: oocytes'], 'age_measurements': ['Measure Type: Count of Participants', 'Unit of measure: Participants'], 'outcome_description': [], 'numberOfPatients': ['18', '22']}


In [303]:
print(crawling_data('NCT03093350'))

1
{'groups': ['TAA-Specific CTLs'], 'treatment': [''], 'ageOfPatients': ['58.80        (11.91)'], 'outcome': ['1'], 'outcome_measurements': ['Measure Type:Count of Participants', 'Unit of Measure: Participants'], 'age_measurements': ['Mean (Standard Deviation)', 'Unit of measure: Years'], 'outcome_description': [], 'numberOfPatients': ['12']}


In [305]:
paloma2

{'StudyType': 'Interventional',
 'InclusionCriteria': ['Woman between the ages of 30 and 65 (both inclusive).',
  'Able to read and understand the Patient Information Sheet and the Informed Consent form.',
  'Acceptance in the participation of the essay and signature of the Informed Consent form.',
  'ASC-US or LSIL or AG-US cytological result (maximum 3 months prior to inclusion) with concordant colposcopic image (performed at the screening visit).',
  'High risk HPV positive by PCR performed at screening visit.',
  'Is able, at the discretion of the researcher, to comply with the requirements of the trial and without hindrance to follow the instructions and assessments throughout it.'],
 'ExclusionCriteria': ['Clinically relevant immune system alterations, or any other autoimmune disease or in treatment with immunosuppressants.',
  'Non-diagnosed abnormal genital hemorrhage (during the 6 months prior to the screening visit).',
  'To had been vaccinated against HPV.',
  'Other symptom

## Complished:
## 1. Takes ~30 seconds to extract all information and create a knowledge graph
## 2. Checked number of columns(groups) before extracting information
## 3. Changed arm/group title from treatment/control groups to their actual names
## 4. Added types and units of age and outcome description directly from the table
## 5. Solve mistakes by different structure:
###   i : //*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[9]/td[2]/div[1] with //*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[9]/td[2]/span
###   ii: count of participants in each age group vs median/mean age (to-do: there will be new properties in knowledge graph about number of patients in each group, currently left blank)


## To-do:
## 1. Extract arm group Description from all p of a td: https://clinicaltrials.gov/ct2/show/results/NCT03093350

## 2. Can't get text directly inside td: 3350

In [109]:
NCT03093350 = dict()
studytype_url = 'https://clinicaltrials.gov/api/query/study_fields?expr=NCT03093350&fields=studytype&min_rnk=1&max_rnk=&fmt=json'
studytype = requests.get(studytype_url).json()['StudyFieldsResponse']['StudyFields'][0]['StudyType'][0]
NCT03093350['StudyType'] = studytype
criteria_url = 'https://clinicaltrials.gov/api/query/full_studies?expr=NCT04176354&min_rnk=1&max_rnk=&fmt=json'
criteria = requests.get(criteria_url).json()['FullStudiesResponse']['FullStudies'][0]['Study']['ProtocolSection']['EligibilityModule']['EligibilityCriteria']

b = criteria.split('\n')
inclusion = []
exclusion = []
for i in range(0,len(b)):
    if b[i] == 'Inclusion Criteria:':
        for j in range(i+1,len(b)):
            if b[j] == 'Exclusion Criteria:':
                break
            if b[j] != '':
                inclusion.append(b[j])
    if b[i] == 'Exclusion Criteria:':
        for j in range(i+1,len(b)):
            if b[j] != '':
                exclusion.append(b[j])
NCT03093350['InclusionCriteria'] = inclusion
NCT03093350['ExclusionCriteria'] = exclusion

drug_url = 'https://clinicaltrials.gov/api/query/study_fields?expr=NCT04176354&fields=ArmGroupInterventionName&min_rnk=1&max_rnk=&fmt=json'
drug = requests.get(drug_url).json()['StudyFieldsResponse']['StudyFields'][0]['ArmGroupInterventionName']
drug
drugs = [i.strip("Drug:").strip("'").strip() for i in drug] 
drugs2 = set()
for i in drugs:
    for j in i.split(" "):
        drugs2.add(j)
# drugs = list(dict.fromkeys(drugs))
drugs2.remove('an')
drugs2.remove('+')
drugs2.remove('aromatase')
drugs2.remove('inhibito')

API_ENDPOINT = 'https://en.wikipedia.org/w/api.php'
drugs_with_des = {}
for i in drugs2:
    drugs_with_des[i] = {}
    query = i
    params = {
        'action':'query',
        'list':'search',
        'srsearch':query,
        'format':'json',
    }
    drugs_with_des[i]['url'] = requests.get(API_ENDPOINT,params=params).url
    drugs_with_des[i]['description'] = requests.get(API_ENDPOINT,params=params).json()\
    ['query']['search'][0]['snippet']
drugs_with_des
NCT03093350['Drugs'] = drugs_with_des



url = "https://clinicaltrials.gov/ct2/show/results/NCT03093350?view=results"
response = urlopen(url)
soup = BS(response, 'html.parser')
baseline = soup.find_all("th",class_="de-popFlowLabelCell")
groups = []
number_of_patients = []
for i in range(0,len(baseline)):
    groups.append(baseline[i].text.strip("\n").strip("\r").strip())
    number_of_patients.append(soup.find_all("td",class_="de-numValue_baselineDataCell")[i].text.strip("\n").strip("\r").strip())

response = urlopen(url)
soup = BS(response, 'html.parser')
dom = etree.HTML(str(soup))


monaleesa2["numberOfPatients"] = []
monaleesa2["numberOfPatients"].append(soup.find_all("td", class_="de-numValue_popFlowCell")[0].text.strip("\r").strip("\n").strip())
monaleesa2["numberOfPatients"].append(soup.find_all("td", class_="de-numValue_popFlowCell")[1].text.strip("\r").strip("\n").strip())
monaleesa2["outcome"] = []
monaleesa2["outcome"].append(soup.find_all("td", class_="de-numValue_outcomeDataCell")[2].text.strip('\n').strip().split(" ")[0])
monaleesa2["outcome"].append(soup.find_all("td", class_="de-numValue_outcomeDataCell")[3].text.strip('\n').strip().split(" ")[0])
monaleesa2["treatment"] = []
monaleesa2["treatment"].append(soup.find_all("td",class_="de-outcomeLabelCell")[7].text.strip('\n').strip('\r').strip())
monaleesa2["treatment"].append(soup.find_all("td",class_="de-outcomeLabelCell")[8].text.strip('\n').strip('\r').strip())
monaleesa2["ageOfPatients"] = []
monaleesa2["ageOfPatients"].append(soup.find_all("td",class_="de-numValue_baselineDataCell")[11].text.strip('\n').strip('\r').strip().split(" ")[0])
monaleesa2["ageOfPatients"].append(soup.find_all("td",class_="de-numValue_baselineDataCell")[12].text.strip('\n').strip('\r').strip().split(" ")[0])
monaleesa2["hazardRatio"] = soup.find_all("td",class_="de-outcomeDataCell")[15].text.strip('\n').strip('\r').strip()

monaleesa2['PFS'] = [dom.xpath('//*[@id="EXPAND-outcome-data-1"]/table/tbody/tr[6]/td[1]/div[1]/text()')[0]
       .strip('\n').strip('\r').strip().replace('\n','').replace('\xa0',''),
                  dom.xpath('//*[@id="EXPAND-outcome-data-1"]/table/tbody/tr[6]/td[2]/div[1]/text()')[0].strip('\n').strip().strip('\n').strip()]
monaleesa2["outcome_measurements"] = [dom.xpath('//*[@id="EXPAND-outcome-data-1"]/table/tbody/tr[5]/th/text()')[0]
       .strip('\n').strip('\r').strip().replace('\n','').replace('\xa0',''),
                             dom.xpath('//*[@id="EXPAND-outcome-data-1"]/table/tbody/tr[5]/th/div/text()')[0]
       .strip('\n').strip('\r').strip().replace('\n','').replace('\xa0','')]
monaleesa2["age_measurements"] = [dom.xpath('//*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[7]/th[1]/div/div[1]/text()')[0]
       .strip('\n').strip('\r').strip().replace('\n','').replace('\xa0',''),
                             dom.xpath('//*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[7]/th[1]/div/div[2]/text()')[0]
       .strip('\n').strip('\r').strip().replace('\n','').replace('\xa0','')]





NCT03093350["numberOfPatients"] = str(dom.xpath('//*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[4]/td/text()')[0]).strip('\n').strip('\r').strip().replace('\n','').replace('\xa0','')
    
NCT03093350["groupTitle"] = str(dom.xpath('//*[@id="tab-body"]/div[1]/div[2]/div[2]/table[2]/tbody/tr[1]/th/text()')[0]).strip('\n').strip('\r').strip().replace('\n','').replace('\xa0','')

NCT03093350["groupDescription"] = str(dom.xpath('//*[@id="EXPAND-armGroupDescriptionRow-baseline"]/td[2]//p/text()'))

NCT03093350["ageOfPatients"] = str(dom.xpath('//*[@id="tab-body"]/div[1]/div[3]/div[2]/table/tbody/tr[9]/td[2]/span/text()')[0]).strip('\n').strip('\r').strip().replace('\n','').replace('\xa0','')

NCT03093350["PFS"] = str(dom.xpath('//*[@id="EXPAND-outcome-data-2"]/table/tbody/tr[6]/td/div[1]/text()')[0]).strip('\n').strip('\r').strip().replace('\n','').replace('\xa0','')    

NCT03093350["PFS_unit"] = str(dom.xpath('//*[@id="EXPAND-outcome-data-2"]/table/tbody/tr[5]/th/div/text()')[0]).strip('\n').strip('\r').strip().replace('\n','').replace('\xa0','') 

NCT03093350["OAS"] = str(dom.xpath('//*[@id="EXPAND-outcome-data-3"]/table/tbody/tr[6]/td/div[1]/text()')[0]).strip('\n').strip('\r').strip().replace('\n','').replace('\xa0','')    

NCT03093350["OAS_unit"] = str(dom.xpath('//*[@id="EXPAND-outcome-data-3"]/table/tbody/tr[5]/th/div/text()')[0]).strip('\n').strip('\r').strip().replace('\n','').replace('\xa0','') 

kg_NCT03093350 = Graph()
kg_NCT03093350.bind('Clinical_Trials', CT)
kg_NCT03093350.bind('schema',SCHEMA)
kg_NCT03093350.add((ct_uri, CT.hasID, Literal('NCT03093350')))
            
ct_uri = URIRef('https://clinicaltrials.gov/ct2/show/NCT03093350')
kg_NCT03093350.add((ct_uri, RDF.type, SCHEMA.MedicalTrial))
title = str(NCT03093350["groupTitle"].replace(' ','_'))
kg_NCT03093350.add((ct_uri, CT.hasGroup, CT[title]))
kg_NCT03093350.add((CT[title], CT.hasDescription, Literal(NCT03093350["groupDescription"])))

kg_NCT03093350.add((ct_uri, CT.hasNumberofPatients, Literal(NCT03093350["numberOfPatients"])))
kg_NCT03093350.add((ct_uri, CT.hasMedianAge, Literal(NCT03093350["ageOfPatients"])))
kg_NCT03093350.add((ct_uri, CT.hasPFS, CT['NCT03093350_PFS']))
kg_NCT03093350.add((CT['NCT03093350_PFS'], CT.hasUnit, Literal(NCT03093350["PFS_unit"])))
kg_NCT03093350.add((CT['NCT03093350_PFS'], SCHEMA.number, Literal(NCT03093350["PFS"])))
kg_NCT03093350.add((ct_uri, CT.hasOAS, CT['NCT03093350_OAS']))
kg_NCT03093350.add((CT['NCT03093350_OAS'], CT.hasUnit, Literal(NCT03093350["OAS_unit"])))
kg_NCT03093350.add((CT['NCT03093350_OAS'], SCHEMA.number, Literal(NCT03093350["OAS"])))  

for i in NCT03093350:
    kg_NCT03093350.add((ct_uri, CT.hasStudyType, Literal(NCT03093350['StudyType'])))
    if i == 'InclusionCriteria':
        for j in NCT03093350[i]:
            kg_NCT03093350.add((ct_uri, CT.hasInclusionCriteria, Literal(j)))
    if i == 'ExclusionCriteria':
        for j in NCT03093350[i]:
            kg_NCT03093350.add((ct_uri, CT.hasExclusionCriteria, Literal(j)))
    if i == 'Drugs':
        for j in NCT03093350[i]:
            k = URIRef(NCT03093350[i][j]['url'])
            kg_NCT03093350.add((ct_uri, CT.hasDrug, k))
            kg_NCT03093350.add((k, SCHEMA.name, Literal(j)))
            kg_NCT03093350.add((k, SCHEMA.description, Literal(NCT03093350[i][j]['description'])))
            
kg_NCT03093350.serialize('new_graphs/NCT03093350.ttl',format="turtle")


In [110]:
NCT03093350["groupDescription"] = str(dom.xpath('//*[@id="EXPAND-armGroupDescriptionRow-baseline"]/td[2]//p/text()'))

NCT03093350["groupDescription"]

'[\'Patients receiving TAA-specific CTLs as therapy for breast cancer.\', \'TAA-specific CTLs: Each patient will receive 2 injections at a fixed dose, 28 days apart, according to the following dose schedule: The expected volume of infusion will be 1 to 10 cc.\', \'Dose schedule:\', \'Day 0: 2 x 10^7 cells/m2\', \'Day 28: 2 x 10^7 cells/m2\', "If patients have stable disease or a partial response by the Response Evaluation Criteria in Solid Tumors (RECIST) criteria at their 6 week evaluation after the 2nd cell dose, they will be eligible to receive up to 6 additional doses of CTLs, At least one month should have passed before each additional dose.. Each additional infusion will consist of the same cell number or less (if there is not enough product available for the subject\'s original dose) than their second infusion. Patients will not be able to receive additional doses until the initial safety profile is completed at 6 weeks following the second infusion."]'

In [ ]:
kg = Graph()
kg.bind('Clinical_Trials', CT)
kg.bind('schema',SCHEMA)
ct_uri = URIRef('https://clinicaltrials.gov/ct2/show/NCT04199078')
kg.add((ct_uri, RDF.type, SCHEMA.MedicalTrial))
kg.add((ct_uri, CT.hasID, Literal('NCT04199078')))
for i in paloma2:
    kg.add((ct_uri, CT.hasStudyType, Literal(paloma2['StudyType'])))
    if i == 'InclusionCriteria':
        for j in paloma2[i]:
            kg.add((ct_uri, CT.hasInclusionCriteria, Literal(j)))
    if i == 'ExclusionCriteria':
        for j in paloma2[i]:
            kg.add((ct_uri, CT.hasExclusionCriteria, Literal(j)))
    if i == 'Drugs':
        for j in paloma2[i]:
            k = URIRef(paloma2[i][j]['url'])
            kg.add((ct_uri, CT.hasDrug, k))
            kg.add((k, SCHEMA.name, Literal(j)))
            kg.add((k, SCHEMA.description, Literal(paloma2[i][j]['description'])))
    
    PFS_field = 'NCT04199078_PFS'
    PFS = 'NCT04199078_PFS_treatmentGroup'
    PFS1 = 'NCT04199078_PFS_controlGroup'
    kg.add((ct_uri, CT.hasPFS,CT[PFS_field]))
    kg.add((CT[PFS_field], CT.hasPFS,CT[PFS]))
    kg.add((CT[PFS], CT.hasGroup, Literal("treatmentGroup")))
    kg.add((CT[PFS], CT.hasUnit, Literal("month")))
    kg.add((CT[PFS], SCHEMA.number, Literal(PFS)))
    kg.add((CT[PFS_field], CT.hasPFS,CT[PFS1]))
    kg.add((CT[PFS1], CT.hasGroup, Literal("controlGroup")))
    kg.add((CT[PFS1], CT.hasUnit, Literal("month")))
    kg.add((CT[PFS1], SCHEMA.number, Literal(PFS1)))
    
    treatment_field = 'NCT04199078_treamentDescription'
    treatment = "NCT04199078_treamentGroupDescription"
    treatment1 = "NCT04199078_controlGroupDescription"
    kg.add((ct_uri, CT.hasTreatmentPlan,CT[treatment_field]))
    kg.add((CT[treatment_field], CT.hasTreatment,CT[treatment]))
    kg.add((CT[treatment], CT.hasGroup, Literal("treatmentGroup")))
    kg.add((CT[treatment], CT.hasdescription, Literal(paloma2["treatment"][0])))
    kg.add((CT[treatment_field], CT.hasTreatment,CT[treatment1]))
    kg.add((CT[treatment1], CT.hasGroup, Literal("controlGroup")))
    kg.add((CT[treatment1], CT.hasdescription, Literal(paloma2["treatment"][1])))
    
    nofP_field = 'NCT04199078_numberofPatients'
    numberofPatients = 'NCT04199078_numberofPatients_treatmentGroup'
    numberofPatients1 = 'NCT04199078_numberofPatients_controlGroup'
    kg.add((ct_uri, CT.hasNumberofPatients,CT[nofP_field]))
    kg.add((CT[nofP_field], CT.hasNumberofPatients,CT[numberofPatients]))
    kg.add((CT[numberofPatients], CT.hasGroup, Literal("treatmentGroup")))
    kg.add((CT[numberofPatients], SCHEMA.number, Literal(paloma2["numberOfPatients"][0])))
    kg.add((CT[numberofPatients], CT.hasUnit, Literal("person")))
    kg.add((CT[nofP_field], CT.hasNumberofPatients,CT[numberofPatients1]))
    kg.add((CT[numberofPatients1], CT.hasGroup, Literal("controlGroup")))
    kg.add((CT[numberofPatients1], SCHEMA.number, Literal(paloma2["numberOfPatients"][1])))
    kg.add((CT[numberofPatients1], CT.hasUnit, Literal("person")))
    
    aofP_field = 'NCT04199078_ageOfPatients_treatmentGroup'
    ageOfPatients = 'NCT04199078_ageOfPatients_controlGroup'
    ageOfPatients1 = paloma2["ageOfPatients"][1]
    kg.add((ct_uri, CT.hasMedianAge,CT[aofP_field]))
    kg.add((CT[aofP_field], CT.hasMedianAge,CT[ageOfPatients]))
    kg.add((CT[ageOfPatients], CT.hasGroup, Literal("treatmentGroup")))
    kg.add((CT[ageOfPatients], SCHEMA.number, Literal(paloma2["ageOfPatients"][0])))
    kg.add((CT[ageOfPatients], CT.hasUnit, Literal("year")))
    kg.add((CT[aofP_field], CT.hasMedianAge,CT[ageOfPatients1]))
    kg.add((CT[ageOfPatients1], CT.hasGroup, Literal("controlGroup")))
    kg.add((CT[ageOfPatients1], SCHEMA.number, Literal(paloma2["ageOfPatients"][1])))
    kg.add((CT[ageOfPatients1], CT.hasUnit, Literal("year")))

#      'hazardRatio': '0.576',
#  'hasPercentageOfPatientsWithVisceralDisease': [48.2, 49.5],
#  'hasPercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy': [48, 49.1],
#  'hasPercentageOfPatientsWithNewlyDiagonosedAdavancedDisease': 37.2}
    PVD_field = 'NCT04199078_percentageofPatientWithVisceraDisease'
    PVD = 'NCT04199078_percentageofPatientWithVisceraDisease_treatmentGroup'
    PVD1 = 'NCT04199078_percentageofPatientWithVisceraDisease_controlGroup'
    kg.add((ct_uri, CT.hasPercentageOfPatientsWithVisceralDisease,CT[PVD_field]))
    kg.add((CT[PVD_field], CT.hasPercentageOfPatientsWithVisceralDisease,CT[PVD]))
    kg.add((CT[PVD], CT.hasGroup, Literal("treatmentGroup")))
    kg.add((CT[PVD], SCHEMA.number, Literal(paloma2["hasPercentageOfPatientsWithVisceralDisease"][0])))
    kg.add((CT[PVD], CT.hasUnit, Literal("percentage")))
    kg.add((CT[PVD_field], CT.hasPercentageOfPatientsWithVisceralDisease,CT[PVD1]))
    kg.add((CT[PVD1], CT.hasGroup, Literal("controlGroup")))
    kg.add((CT[PVD1], SCHEMA.number, Literal(paloma2["hasPercentageOfPatientsWithVisceralDisease"][1])))
    kg.add((CT[PVD1], CT.hasUnit, Literal("percentage")))
    
    PriorChemo_field = 'NCT04199078_percentageofPatientsHadReceivedPriorAdjuvantChemotherapy'
    PercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy = 'NCT04199078_percentageofPatientsHadReceivedPriorAdjuvantChemotherapy_treatmentGroup'
    PercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy1 = 'NCT04199078_percentageofPatientsHadReceivedPriorAdjuvantChemotherapy_controlGroup'
    kg.add((ct_uri, CT.hasPercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy,CT[PriorChemo_field]))
    kg.add((CT[PriorChemo_field], CT.hasPercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy,
            CT[PercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy]))
    kg.add((CT[PercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy], CT.hasGroup, Literal("treatmentGroup")))
    kg.add((CT[PercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy], SCHEMA.number, Literal(paloma2["hasPercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy"][0])))
    kg.add((CT[PercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy], CT.hasUnit, Literal("percentage")))
    kg.add((CT[PriorChemo_field], CT.hasPercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy,
            CT[PercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy1]))
    kg.add((CT[PercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy1], CT.hasGroup, Literal("controlGroup")))
    kg.add((CT[PercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy1], SCHEMA.number, Literal(paloma2["hasPercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy"][1])))
    kg.add((CT[PercentageOfPatientsHadReceivedPriorAdjuvantChemotherapy1], CT.hasUnit, Literal("percentage")))    
    
    hazardRatio = paloma2["hazardRatio"]
    kg.add((ct_uri, CT.hasHazardRatio, CT['NCT04199078_hazardRatio']))
    kg.add((CT['NCT04199078_hazardRatio'], SCHEMA.number, Literal(paloma2["hazardRatio"])))
    kg.add((CT['NCT04199078_hazardRatio'], CT.hasConfidenceInterval, Literal("(2 sided)95% 0.463 to 0.718")))
    
    PercentageOfPatientsWithNewlyDiagonosedAdavancedDisease = str(paloma2["hasPercentageOfPatientsWithNewlyDiagonosedAdavancedDisease"])
    kg.add((ct_uri, CT.hasPercentageOfPatientsWithNewlyDiagonosedAdavancedDisease, CT['NCT04199078_PercentageOfPatientsWithNewlyDiagonosedAdavancedDisease']))
    kg.add((CT['NCT04199078_PercentageOfPatientsWithNewlyDiagonosedAdavancedDisease'], SCHEMA.number, Literal(paloma2["hasPercentageOfPatientsWithNewlyDiagonosedAdavancedDisease"])))
    kg.add((CT['NCT04199078_PercentageOfPatientsWithNewlyDiagonosedAdavancedDisease'], CT.hasUnit, Literal("percentage")))
    
kg.serialize('paloma-2.ttl', format="turtle")    